# Data Loading, Cleaning and Wrangling

In [1]:
#Need to run this to have the right path
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Installing the right stuff
!pip install -q sentencepiece
!pip install -q transformers
!pip install -q evaluate
!pip install -q rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
#Importing necessary libraries for cleaning
import pandas as pd
import numpy as np
import evaluate
from pprint import pprint

In [4]:
#Building path and opening the files to create the dataframe
data_path = "/content/drive/MyDrive/266 NLP - Final Project/all_the_news_SUBSET_Kaggle/"

df1 = pd.read_csv(data_path + "articles1.csv").iloc[:, 1:]
df2 = pd.read_csv(data_path + "articles2.csv").iloc[:, 1:]
df3 = pd.read_csv(data_path + "articles3.csv").iloc[:, 1:]

data = pd.concat([df1, df2, df3])

In [5]:
### Data Characteristics
## Each id is unique to each row/headline with no duplicate articles/rows
## 142,570 total headline and articles

## 1.85% of the data doesn't have dates but I feel like it's ok to get rid of them since it's such a small sample
data[data["date"].isnull()].shape[0]/data.shape[0]
data = data[~data["date"].isnull()]

## Out of all the articles, 99.7% of them are from 2015-2017 so it might not be helpful to keep the older articles before 2015 since there are so few
data.value_counts("year").sort_index(ascending=False)
data = data[data["year"]>2014]

data = data.reset_index()

#Added word count column
data['article_wd_ct'] = data['content'].str.split().str.len() ## word count of each article

## Only subsetting for article that have at least 100 words- could do more analysis into including ones between 70-100 but the ones I've seen in that range aren't fully populated articles or poorly extracted articles
data = data[data["article_wd_ct"]>=100]

## Further subsetting the data for articles below 500 words for ease of running the models
data = data[data["article_wd_ct"]<=500]

Data Cleaning for Headlines.

A manual process done publication by publication

In [6]:
#Atlantic
data = data.drop(data.loc[(data.publication=="Atlantic") & (data.title.str.contains("Atlantic"))].index)

#Breitbart
data.loc[((data.publication=="Breitbart") & (data.title.str.contains("- Breitbart"))), "title"] = data[(data["publication"] == "Breitbart") & (data["title"].str.contains("- Breitbart"))].title.str[:-12]
data = data.drop(data.loc[(data.publication=="Breitbart") & ( (data.title.str.contains("Breitbart")) | (data.title.str.contains("BREITBART")) )].index)

#Business Insider
data = data.drop(data.loc[(data.publication=="Business Insider") & ((data.title.str.contains("Business Insider")) | (data.title.str.contains("BUSINESS INSIDER")) )].index)

#BuzzFeed
data = data.drop(data.loc[(data.publication=="Buzzfeed News") & ((data.title.str.contains("BuzzFeed")) | (data.title.str.contains("BUZZFEED")) )].index)

#CNN
data = data.drop(data.loc[(data.publication=="CNN") & (data.title.str.contains("CNN"))].index)

#Fox News
#Fix all the aforementioned appendings. Data frame should be equal size to above, 53089, can use that to validate.
data.loc[((data.publication=="Fox News") & (data.title.str.contains("Fox News Poll:"))), "title"] = data[(data["publication"] == "Fox News") & (data["title"].str.contains("Fox News Poll:"))].title.str[15:]
data.loc[((data.publication=="Fox News") & (data.title.str.contains("Fox News Electoral Scorecard:"))), "title"] = data[(data["publication"] == "Fox News") & (data["title"].str.contains("Fox News Electoral Scorecard:"))].title.str[30:]
data.loc[((data.publication=="Fox News") & (data.title.str.contains("Fox News projects:"))), "title"] = data[(data["publication"] == "Fox News") & (data["title"].str.contains("Fox News projects:"))].title.str[19:]
data = data.drop(data.loc[(data.publication=="Fox News") & ((data.title.str.contains("Fox")) | (data.title.str.contains("FOX")) )].index)

#Guardian
data = data.drop(data.loc[(data.publication=="Guardian") & (data.title.str.contains("Guardian"))].index)

#National Review
data = data.drop(data.loc[(data.publication=="National Review") & ( (data.title.str.contains("NR ")) | (data.title.str.contains("NR:")) | (data.title.str.contains("NRI")) | (data.title.str.contains("NRO")) )].index)

#New York Post
data = data.drop(data.loc[(data.publication=="New York Post") & (data.title.str.contains("New York Post"))].index)

#New York Times
data.loc[data.publication=="New York Times", "title"] = data[data['publication'] == "New York Times"].title.str[:-21]
data = data.drop(data.loc[(data.publication=="New York Times") & (data.title.str.contains("New York Times"))].index)

#NPR
data = data.drop(data.loc[(data.publication=="NPR") & (data.title.str.contains("NPR"))].index)

#Reuters
data = data.drop(data.loc[(data.publication=="Reuters") & (data.title.str.contains("Reuters"))].index)

#Talking Points Memo
data = data.drop(data.loc[(data.publication=="Talking Points Memo") & (data.title.str.contains("TPM"))].index)

#Vox
data = data.drop(data.loc[(data.publication=="Vox") & (data.title.str.contains("Vox"))].index)

#Washington Post
data = data.drop(data.loc[(data.publication=="Washington Post") & (data.title.str.contains("Washington Post"))].index)

In [7]:
#Totals got iffy, but it's at 52958
data.reset_index()

#data

#Do not uncomment unless you want to download
#data.to_excel("CLEANED compact.xlsx")

,level_0,index,id,title,publication,author,date,year,month,url,content,article_wd_ct
0,5,5,17288,"Sick With a Cold, Queen Elizabeth Misses New Y...",New York Times,Sewell Chan,2017-01-02,2017.0,1.0,NaN,"LONDON — Queen Elizabeth II, who has been b...",159
1,19,19,17303,Fecal Pollution Taints Water at Melbourne’s Be...,New York Times,Brett Cole,2017-01-03,2017.0,1.0,NaN,"SYDNEY, Australia — The annual beach pilgri...",292
2,33,33,17323,"Airline Pilot, Believed to Be Drunk, Is Pulled...",New York Times,Ian Austen,2017-01-03,2017.0,1.0,NaN,OTTAWA — It was 7 a. m. and 99 passengers a...,410
3,47,47,17338,It’s Time to Ignore Advice About Which Stocks ...,New York Times,Damon Darlin,2017-01-09,2017.0,1.0,NaN,It’s that time of year when financial advice s...,395
4,52,52,17344,Chinese City Official Shoots 2 Others and Kill...,New York Times,Chris Buckley and Adam Wu,2017-01-05,2017.0,1.0,NaN,BEIJING — A city official in southwest Chin...,358
...,...,...,...,...,...,...,...,...,...,...,...,...
52953,139478,42535,218042,Debbie Reynolds was a trouper all the way to t...,Washington Post,Ann Hornaday,2016-12-29,2016.0,12.0,https://web.archive.org/web/20161230000842/htt...,Y ou can almost hear Carrie Fishe...,367
52954,139487,42544,218051,An obscure E.U. regulation may have saved live...,Washington Post,Adam Taylor,2016-12-29,2016.0,12.0,https://web.archive.org/web/20161230000842/htt...,The truck attack on a Christmas market in...,387
52955,139492,42549,218056,A woman ordered canaries to brighten her home....,Washington Post,Avi Selk,2016-12-29,2016.0,12.0,https://web.archive.org/web/20161230000842/htt...,"Somewhere between Texas and Alabama, some...",373
52956,139494,42551,218063,"Trump praises Putin’s response to sanctions, c...",Washington Post,Karoun Demirjian,2016-12-30,2016.0,12.0,https://web.archive.org/web/20161231004909/htt...,Donald Trump on Friday expressed his ap...,314


In [8]:
headlines_rand = ['Donald Trump Takes Credit for Helping to Save a Ford Plant That Wasn’t Closing',
 '‘Ring of Fire’ Eclipse Travels Across South America and Africa',
 'Bruce Springsteen’s Archive Is Headed to Monmouth University',
 'California Official Says Trump’s Claim of Voter Fraud Is ‘Absurd’',
 'Monkey in Kenya Survives After Setting Off Nationwide Blackout',
 'Hillary Clinton Warns of ‘Conspiracy Theory Machine Factory’',
 'Report: GOP Differences on Healthcare ’Have Narrowed,’ Healthcare Reform ’Very Much Alive’',
 'Donald Trump Appoints Top Deputies, More White House Staff',
 'LISTEN: Green Day Single Focuses on American ’Culture of Mass Shooting’',
 'Five Facts About Donald Trump’s Expected Pick for Interior Secretary, Rep. Cathy McMorris Rodgers',
 'George W. Bush to attend fundraiser in support of Gillespie',
 'Justin Timberlake and Jessica Biel expecting first baby',
 'Bill Cosby drops lawsuit against accuser',
 'Lupita Nyong’o’s dress possibly recovered',
 'New Trump accuser: GOP nominee grabbed, kissed me',
 'Wealthy shoppers’ latest habit is killing Restoration Hardware',
 'Trump campaign says it had ’nothing to do with’ him appearing on Kremlin-backed Russian news outlet',
 'Silicon Valley’s favorite coffee chain is taking over the US with an east coast expansion',
 'ANN COULTER: Here’s who Donald Trump should pick as his running mate',
 'Snapchat’s CEO wore the company’s secret-camera sunglasses in public — and nobody noticed',
 'The Rescuing of 6,000 Migrants in the Mediterranean',
 'The Lost 16th-Century Spanish Fort in South Carolina',
 'Another Flogging for Wells Fargo',
 'Why Did It Take ISIS a Full Day to Claim the London Attack?',
 'The World’s ‘Largest Resettlement Effort’ for Child Refugees',
 'Photo of Obama in Muslim garb shows deep ties to faith, O’Reilly says',
 'Facebook activates Safety Check service after Orlando nightclub shooting',
 ' Approval of Obama up, US weaker under his leadership',
 'Coast Guard searching for woman who fell off cruise ship in Gulf of Mexico',
 'Hurricane Matthew unearths huge prehistoric shark tooth',
 'Drifting Down a River in Egypt',
 'Students Charged With Hate Crime For Painting Swastika, ’Trump’ On Chapel',
 'Cruz Calls On Democrats To Support His Campaign In Iowa Victory Speech',
 'Professor Who Wore Headscarf To Leave Christian College',
 'Fiorina Wants The RNC To Demand She Get A Spot In New Hampshire Debate',
 ' Actor Abe Vigoda Dies At\xa094',
 ' Things Go From Bad To Worse On The Great Barrier\xa0Reef',
 ' Piers Morgan Was Told To “Fuck Off” On Live TV For Saying Trump’s Ban Wasn’t Targeting\xa0Muslims',
 ' Read Tim Cook’s Email To Apple Employees After Donald Trump’s\xa0Election',
 ' Kanye’s Album Is Finally Out And Here’s How You Can\xa0Listen',
 'Trump, Alack',
 'Disney Accused of Cultural Insensitivity Because a Character Is Too Fat ',
 'Three Cups of Weak Tea',
 'Student Group Sues School for Allowing a Social-Media App Where People Posted Mean Stuff',
 'Top Divinity Schools: Use Gender-Neutral Language to Refer to God',
 'Over 1K city bodega owners plan to strike in protest of travel\xa0ban',
 'Qatar buys a piece of Empire State\xa0Building',
 'MetLife’s ‘too-big-to-fail’ fight with feds gets off to good\xa0start',
 'Tony Romo taking Phil Simms’ spot at\xa0CBS',
 'Ex-Yankee Chad Curtis near deal with girls he\xa0touched',
 'Google’s self-driving car in broadside collision after other car jumps red light',
 'First look at Rihanna as Marion Crane in new Bates Motel trailer',
 'William Tyler: Modern Country review – Americana too sublime for words',
 'Norway: ’bomb-like device’ destroyed in central Oslo',
 '’Very weak’: Donald Trump blasts Republican senator Jeff Flake',
 'Syd’s Solo Debut ’Fin’ Can Smoke Out The Quiet Storm',
 'To Save Power, Venezuela Gives Public Employees Fridays Off',
 'Shinzo Abe Will Be First Japanese Leader To Visit Pearl Harbor Memorial',
 'Pregnant Women May Be Able To Get Answers About Zika Earlier',
 'Vatican: Don’t Scatter Cremation Ashes, And Don’t Keep Them At Home',
 'American Airlines wins $15 million in antitrust case against Sabre',
 'Fincantieri’s bid for STX France approved by South Korean court',
 '’I had to do it,’ accused gunman says of South Carolina church attack',
 'Obama rallies Obamacare troops at ’critical time’ for program',
 'Viewsroom: What to watch for beyond Clinton-Trump',
 'This is probably the biggest tax you pay. No one talks about it.',
 'Florida tried to ban discrimination against LGBTQ people — and it failed',
 '9 bonkers things Donald Trump said in his new exclusive interview with Time',
 'Bernie defended Clinton on her leaked emails — but he couldn’t resist taking one shot',
 'Donald Trump’s new regulation czar is a conflict-of-interest disaster',
 'Senate Intelligence Committee requests Trump campaign documents',
 '‘Donald Trump destroyed my life,’ says barred Iraqi who worked for U.S.',
 'These 8th-graders from New Jersey refused to be photographed with Paul Ryan',
 'Donald Trump set to be deposed in José Andrés case just before inauguration',
 '‘You’ve Got a Friend’: Singers who charted in the ’70s raise the volume on Clinton fundraising']

In [9]:
articles_rand = ['WASHINGTON  —     Donald J. Trump claimed credit on Thursday night for persuading Ford to keep an automaking plant in Kentucky rather than moving it to Mexico. The only wrinkle: Ford was not actually planning to move the plant. Mr. Trump wrote on Twitter shortly after 9 p. m. that Ford’s chairman, William Clay Ford Jr. had just told him that Ford “will be keeping the Lincoln plant in Kentucky  —   no Mexico. ” Minutes later, Mr. Trump wrote in a second post: “I worked hard with Bill Ford to keep the Lincoln plant in Kentucky. I owed it to the great State of Kentucky for their confidence in me!” Mr. Trump won 62. 5 percent of the state’s popular vote in the presidential election. During the campaign, he repeatedly criticized Ford for moving production to Mexico, and he threatened to impose a 35 percent tariff on vehicles made there. Ford makes the Lincoln MKC, a sport utility vehicle, at a factory in Louisville. Last week, Ford said it planned to move production of the vehicle elsewhere. On Thursday night, after Mr. Trump’s Twitter messages, the company said that Mexico had been the intended destination and that it would now keep MKC production in Kentucky. But Ford had not planned to close the Louisville factory. Instead, it had planned to expand production of another vehicle made in Louisville, the Ford Escape. And the change had not been expected to result in any job losses. “Whatever happens in Louisville, it will not lose employment,” Jimmy Settles, a union official, told The Detroit Free Press. “They cannot make enough Escapes. ” Now, thanks to Mr. Trump, the plant will make fewer Escapes  —   and more MKCs. “We are encouraged that   Trump and the new Congress will pursue policies that will improve U. S. competitiveness and make it possible to keep production of this vehicle here in the United States,” Ford said in a statement.',
 'People in parts of South America and Africa were treated to a rare sight on Sunday: a “ring of fire” eclipse, when the moon moves in between the Earth and the sun, briefly replacing it with a blazing, fiery ring. An event of this kind  —   technically called an annular eclipse  —   is a byproduct of the moon’s elliptical nature around the Earth, which means it is sometimes further away from the planet than at other times. An annular eclipse is one that happens when the moon is unable to completely block our view of the sun because it is at one of its greater distances from Earth. The countries with the best chance of viewing an annular eclipse are those that lie along the path of annularity, an invisible line that traces the path taken by the moon’s shadow as it moves across the globe, according to C. Alex Young, a solar astrophysicist from NASA. They include Chile and Argentina in South America and Angola, Zambia and the Democratic Republic of Congo in Africa, where viewers were able to see the full “ring of fire” effect for roughly one minute. Skywatchers on the ground in those countries  —   including more than a few government agencies  —     their experience on Sunday, with some complaining that overcast weather dampened their views. People outside the direct path of annularity could still get a glimpse of the show although it was only be a partial eclipse. For everyone else, there is always the internet. Slooh Community Observatory, a system of telescopes, broadcast the eclipse online via live stream beginning around 7 a. m. Eastern time on Sunday. Americans will also get to watch an eclipse of their own soon enough. A total solar eclipse is due on Aug. 21.',
 'There really could be only one appropriate home for Bruce Springsteen’s archives: the Jersey Shore. And indeed that is where they will go, through a partnership announced Tuesday with Monmouth University in West Long Branch, N. J. As part of the partnership, the university will establish the Bruce Springsteen Archives and Center for American Music, which will be the repository for Mr. Springsteen’s personal collection of written works, photographs, periodicals and various artifacts from throughout his career. The university  —   just miles from Asbury Park, one of the towns where Mr. Springsteen started his musical career  —   said in a statement that the new center would promote the legacy of Mr. Springsteen and other giants of American music, like Woody Guthrie and Robert Johnson. Its materials would also bolster curriculums at the university, including at its music business program. “The establishment of the Bruce Springsteen Archives and Center for American Music celebrates and reinforces the Jersey Shore’s legacy in the history of American music, while providing a truly transformative experience for our students,” Paul R. Brown, the university’s president, said in a statement. The university offered few other details about the collection or its plans for the new center, including any financial information about the partnership. But Mr. Springsteen’s materials will join what is already a major trove of memorabilia at Monmouth, the Bruce Springsteen Special Collection. That collection includes nearly 35, 000 items  —   compiled in part by fans  —   which has been housed at Monmouth since 2011. One of the figures involved in bringing the archives to Monmouth University was Robert Santelli, the executive director of the Grammy Museum in Los Angeles, who is expected to take on a leadership role at the new center. Mr. Santelli, a Monmouth alumnus, also helped secure the special collection for the university in 2011. The arrangement with Monmouth comes as   archives have become increasingly valuable to museums, universities and other cultural institutions, which use them for scholarly study and sometimes as tourist attractions. Last year, Bob Dylan’s archives were acquired by the George Kaiser Family Foundation for a group of institutions in Oklahoma, including the University of Tulsa, for an estimated $15 million to $20 million.',
 'LOS ANGELES  —   When   Donald J. Trump claimed on Twitter that he was losing the popular vote because of major fraud by millions of voters, one of the states he pointed to was California, where the latest voting returns showed Hillary Clinton, the Democrat, crushing Mr. Trump. But Mr. Trump’s baseless claim led to a furious reaction from California’s top election official, Alex Padilla, the secretary of state, this weekend. Mr. Padilla asserted that there was no evidence for the claim by the   and denounced Mr. Trump for what he said was unpresidential behavior. “His unsubstantiated allegations of voter fraud in California and elsewhere are absurd,” Mr. Padilla posted on Twitter. “His reckless tweets are inappropriate and unbecoming of a  . ” This state has historically been slow to count ballots, a reflection of both its vast size and inefficiencies in many county voting operations. Given the fact that this is an overwhelmingly Democratic state, that has meant that Mrs. Clinton’s total vote count has grown steadily as ballots were tallied, adding to a national lead of close to two million votes. As of Saturday, Mrs. Clinton had 8. 1 million votes in California, compared with 4. 2 million for Mr. Trump, according to the secretary of state’s office. It was not clear when the vote count might be concluded. The count, and outcome, has been no surprise to anyone in a state with a history of slow  . Officials in both parties had predicted this would happen as early as election night. Given California’s long Democratic history, it was never in play during this presidential election. But as the nation’s most populated state, it tends to have a significant influence on final national voting margins, as is apparently the case this year. Mr. Trump signaled out three states in his post of Twitter on “serious voter fraud”  —   Virginia, New Hampshire and California. He offered no evidence to back up the claim. His remarks came as he denounced calls for a recount in three states that he won by relatively small margins: Wisconsin, Michigan and Pennsylvania. Jill Stein, who was the Green Party candidate for president, said that she would move for a recount. Aides to Mrs. Clinton said they would cooperate with the effort, even as they made it clear they thought it would not change the outcome. Mr. Padilla is the   Latino elected to state office in California. Mr. Trump’s poor showing here, many Democrats and Republicans said, came in no small part because of his attacks on what he described as the threat of illegal immigration  —   particularly by Mexicans. About 40 percent of California’s population is Latino. “It appears that Mr. Trump is troubled by the fact that a growing majority of Americans did not vote for him,” Mr. Padilla said.',
 'NAIROBI, Kenya  —   Kenyan consumers have long suspected a little monkey business when it comes to their power bills, but the news making its way around the country on Wednesday was almost too much to believe. A nationwide blackout that knocked out lights, severed internet service and paralyzed countless businesses for more than three hours on Tuesday was caused by a monkey, the electricity company announced. The monkey, which appeared to be of the Vervet species, with a collar of white fur, climbed onto the roof of the Gitaru Power Station and fell or jumped onto a transformer. The transformer tripped, which in turn tripped other machines at the hydroelectric power station, setting off a nationwide blackout, the Kenya Electricity Generation Company, or KenGen, said. “KenGen power installations are secured by electric fencing, which keeps away marauding wild animals,” the company said. “We regret this isolated incident and the company is looking at ways of further enhancing security at all our power plants. ” The monkey survived and was given to Kenya’s wildlife services, KenGen said. The company posted a photograph on Facebook showing the animal on the transformer, crouching shyly. Vervet monkeys, which are usually about two feet long, not including the tail, can be pesky creatures. They can be seen everywhere in Kenya, and they often travel in packs, scampering along power lines, snacking on mangoes in backyards and even sneaking into kitchens through open windows to pinch a banana or two. KenGen, which has been criticized over what some call Kenya’s high electricity costs, said late Tuesday that the failure had been fixed. Some homes in Nairobi were still without power on Wednesday.',
 'Democratic presidential nominee Hillary Clinton dismissed questions about her F. B. I. testimony that was released over the weekend, insisting that they were part of a “conspiracy theory machine factory. ”[“I believe I’ve created so many jobs in the story of conspiracy theory machine factory, because honestly they never quit. They keep coming back. ” she said.  She said the F. B. I. had already investigated her email habits and that the report “debunked” all of the additional lies that were exposed as a result. “Their report answered all the questions,” she insisted. “The findings included debunking his latest conspiracy theories. ” She also complained that too many Republicans focused on “partisan conspiracy theorizing” instead of working together to solve problems.',
 'On Thursday’s broadcast of CNN’s “At This Hour,” CNN National Political Reporter MJ Lee reported that House members are working on a deal on a healthcare bill “that could potentially bring on board 18 to 20 new yes votes from the House Freedom Caucus. ” And Representative Tom Cole ( ) told her “the differences have narrowed, and this thing is very much alive. ”  Lee said, “So, there are signs of a possible breakthrough on healthcare, emphasis on possible. What we know is that leaders of the Tuesday Group, as well as the House Freedom Caucus have been in talks over the easter recess to try to revive the first failed healthcare bill that was pulled from the House floor. Now, what the members are trying to work on right now is a deal that could potentially bring on board 18 to 20 new yes votes from the House Freedom Caucus. Now, I just spoke a little while ago with Republican Congressman Tom Cole, and he tells me that the differences have narrowed, and this thing is very much alive. ” She added, “Now, there are some serious obstacles heading into next week. As you know, Kate, members have to deal with the problem of funding the government, and that is sort of their first priority. And we also know that the existing splits between the conservative faction, as well as the moderates, that continues to sort of be a problem as they try to get to the 216 votes. So, we’ll see if any sort of real breakthrough comes before members come back next week. ” (  RCP Video) Follow Ian Hanchett on Twitter @IanHanchett',
 '  Donald Trump has appointed former RNC Chief of Staff Katie Walsh to serve as his Deputy Chief of Staff to the White House, according to a release from the transition team. [He also named Rick Dearborn, a chief of staff for Sen. Jeff Sessions, to serve as the Deputy Chief of Staff for Legislative and Intergovernmental Affairs, and former Bush administration official Joe Hagin as the Deputy Chief of Staff for Operations at the White House.  “Rick and Katie have been instrumental in our victory and transition efforts,” said Trump’s Chief Strategist Steve Bannon in a statement. “These leaders will guide critical White House operations as we aggressively follow through on   Trump’s promise to put America First. ” The appointments also drew praise from incoming Chief of Staff Reince Priebus and Kellyanne Conway, the recently named Counselor to the President. “Katie Walsh is an ” Conway said in a statement. “She helped our party achieve sweeping victories across the country, and I’m thrilled to have another strong female leader on our Senior White House team. ” “The  ’s bold vision to make America great again demands a team of doers who can hit the ground running on day one,” Priebus said in a statement. “Katie, Rick, and Joe will fill critical roles as we push forward on all of the key priorities for our great nation. ” The Trump transition team also released a list of hires to assist the president at the White House: Marc Short, Assistant to the President and Director of Legislative AffairsJohn DeStefano, Assistant to the President and Director of Presidential PersonnelOmarosa Manigault, Assistant to the President and Director of Communications for the Office of Public LiaisonJosh Pitcock, Assistant to President and Chief of Staff to the Vice PresidentKeith Schiller, Deputy Assistant to the President and Director of Oval Office OperationsGeorge Gigicos, Deputy Assistant to the President and Director of AdvanceJessica Ditto, Deputy Assistant to the President and Deputy Communications DirectorRaj Shah, Deputy Assistant to the President and Deputy Communications Director and Research DirectorBill Stepien, Deputy Assistant to the President and Political DirectorJen Pavlik, Deputy Assistant to the President and Deputy Chief of Staff to the Vice PresidentJohn McEntee, Special Assistant to the President and Personal Aide to the President',
 'Rock band Green Day has returned after a   hiatus  —   and their new single “Bang Bang” is all about mass shootings and the “narcissistic social media” coverage that the tragic events often engender. [Green Day frontman Billie Joe Armstrong told Rolling Stone the song is representative of changes in culture in which people who carry out heinous acts receive celebrity status, sometimes even filming themselves committing the act and disseminating it before their death or capture.  “There’s this sort of rage happening, but it’s also now being filmed and we all have ourselves under surveillance. To me, that is so twisted,” Armstrong told the outlet.  Armstrong told Rolling Stone he was inspired to write the group’s forthcoming album, Revolution Radio, after driving thorough Manhattan and witnessing demonstrators protesting the shooting death of Ferguson, Missouri teenager Michael Brown. “I got out of my car and marched with the people,” he said. “It was a trip to see people rebel against the old order. ” The album’s lead single, “Bang Bang,” delves into the perspective of a mass shooter. “I wouldn’t even say I was trying to understand it,” Armstrong told RS. “I was just trying to figure out the character. I don’t know why someone would ever do something that horrific because I know I never would. It’s just sort of meant to also reflect the culture a little bit without sounding pretentious. ” Nevertheless, Armstrong told the outlet he remains “optimistic” about the future of the country. “A lot of what people cover on sort of the corporate news outlets is the sh*t show. And it’s the reality TV show of like, ‘Look at what this election has turned into.’ But look at the other things and movements that are happening,” the singer said. “I think Bernie Sanders broke new ground, not just as a protest candidate, but he broke into the inside of Washington,” he added. “I think that getting these young people to start voting and   and running for office in their own towns   …     I think that the next 10 years is going to be a big  . ” Listen to Green Day’s “Bang Bang” above.   AWR Hawkins is the Second Amendment columnist for Breitbart News and political analyst for Armed American Radio. Follow him on Twitter: @AWRHawkins. Reach him directly at awrhawkins@breitbart. com.      ',
 'Donald Trump is expected to pick Rep. Cathy McMorris Rodgers as his Interior Secretary, making her a central player in his plans to   the domestic oil, natural gas and coal industries. [She was first elected in 2004, and is now the   person in the GOP’s House leadership, under House Speaker Paul Ryan.  She also had time to birth three children while in Congress. Here’s her Instagram account, and a cute picture of her balancing work and  .  I’ve got my #SOTU remarks in one hand and   old Brynn in another. It really doesn’t get much better than this! A photo posted by Cathy McMorris Rodgers (@cathymcmorris) on Jan 24, 2014 at 11:15am PST,  Feminists and   don’t like her conservative social record. According to an respectful but unfriendly profile in Slate. com, McMorris Rodgers has voted like a standard conservative, for cuts to nearly every social service. She voted against the Lilly Ledbetter Fair Pay Act and in favor of cutting funding for birth control. Last year, she supported a version of the Violence Against Women Act that excluded gay, immigrant, and Native American women, calling them a “side issue. ” And her constituents once delivered empty milk bottles to her office to protest her support for cuts in the “WIC” nutritional program. The war on women, Matt Yglesias points out, is not about symbols. It’s about public policy. So if you want something like that done, ask another woman. McMorris Rodgers is from Washington state, where her constituents include Microsoft Corp. and apple growers, and so she has strongly backed   immigration of   graduates and unskilled, illiterate  . She supported the 2013 “Comprehensive Immigration Reform” push, which failed in June 2014 when GOP primary voters demonstrated their opposition by defeating GOP majority Leader Rep. Eric Cantor. In 2011, she   a bill to sell off 0. 6 percent of the 500 million acres of   land. Those holdings add up to 20 percent of the nation’s  . She gets little love from   environmental groups, because she generally favors drilling, mining and logging on the huge tracts of   land in the western half of the United States. She will also have a huge say on the construction of oil and   pipelines.        ',
 ' (CNN) Former President George W. Bush will attend a fundraiser this weekend in support of a Republican candidate for governor of Virginia, Ed Gillespie. The fundraiser will take place in Dallas, where the former president lives, and is expected to draw many   donors. Gillespie served as a top aide to Bush during his time in the White House and was chairman of the Republican National Committee. The former president has largely stayed away from politics since leaving office. He has rarely campaigned or raised money for any candidates aside from his brother, former Florida Gov. Jeb Bush. ”Ed will travel to Dallas this week to raise money for his campaign. He’s proud to have the support of President Bush and conservative leaders from across the commonwealth and the country,” said Abbi Sigler, a Gillespie spokeswoman. ”This is a crucial campaign for Virginia Republicans, and Ed will work tirelessly to ensure we have the resources necessary to defeat Ralph Northam or Tom Perriello in November, and bring conservative leadership back to Richmond.” The fundraiser comes at a time when Gillespie has pulled away from the pack in a field of Republicans hoping to secure the GOP nomination. A new poll from Christopher Newport University shows Gillespie with a sizable lead in the primary over Corey Stewart, the chair of the Prince William County Board of Supervisors and State Senator Frank Wagner. However, more than a third of Republicans    38%    remain undecided. The Roanoke Times first reported the news of the fundraiser. Stewart once served as the chair of Donald Trump’s presidential campaign in Virginia before being fired. His   campaign which includes tagging Gillespie with the moniker ”Establishment Ed” because of his lengthy career in Washington has not yet caught on. Democrats have a fierce primary battle of their own in Virginia. Lt. Gov. Ralph Northam, who enjoys the support of all of Virginia’s top elected democratic officials, is locked in a tight battle with former Congressman Tom Perriello. That same CNU poll shows the Democratic race is a tie. While Northam has the support of the Virginia Democratic infrastructure behind him, led by Gov. Terry McAuliffe, Perriello has pitched himself as the more authentic progressive    highlighted by the revelation in The New York Times that before Northam became a candidate he voted for George W. Bush twice. ',
 ' (CNN) Justin Timberlake is expecting ”the greatest gift ever” this year: a baby. After months of speculation, the singer confirmed Saturday that his wife, Jessica Biel, is expecting their first child. News of the impending arrival came on his 34th birthday.  In a social media post, Timberlake said they don’t know the sex of the baby yet, but are taking bets. The post included a picture of him lovingly cradling and kissing what appears to be his wife’s baby bump. ”Thank you EVERYONE for the Bday wishes! This year, I’m getting the GREATEST GIFT EVER. CAN’T WAIT,” Timberlake said on Instagram.  Biel has appeared in various movies, including ”Valentine’s Day” and the TV series ”7th Heaven.”  The two were married in southern Italy in 2012, but rarely reveal information about their relationship.  In October, Timberlake posted an Instagram picture of them sitting on a bench near a lake,  providing a rare insight into their life.   ”’Life moves pretty fast. If you don’t stop and look around once in a while, you could miss it,” he posted under the picture, using a quote from the movie ”Ferris Bueller’s Day Off.”  Biel has been pictured with a baby bump lately, but this is the first time the couple has confirmed it.   ',
 ' (CNN) Bill Cosby has dismissed a federal lawsuit he filed earlier this year against Andrea Constand, his accuser in an upcoming criminal trial, according to a recently released document. The suit had also named Constand’s mother, attorneys Dolores Troiani and Bebe Kivitz and American Media Inc. owner of the National Enquirer.  In the lawsuit, Cosby alleged the defendants violated a 2006 agreement that settled civil suits Constand had filed against the comedian and American Media. Cosby said Constand and her attorney Troiani voluntarily cooperated with the Montgomery County district attorney’s criminal investigation into her   sexual assault allegation against the actor and comedian. The 2006 settlement ”expressly prohibited” such cooperation, according to Cosby’s lawsuit. Angela Agrusa, an attorney representing the comedian, said Cosby ”has today stepped away from that suit and will instead focus his efforts on defending himself against the claims that have been lodged against him.” The voluntary dismissal action was filed Thursday in U. S. District Court for the Eastern District of Pennsylvania. Cosby faces a criminal trial on three counts of felony aggravated indecent assault from a 2004 case involving Constand, an employee at his alma mater, Temple University. He denies all the charges. Troiani said the dismissal of the lawsuit was a win for Constand and her mother, Gianna Constand. ”Andrea and Gianna stood firm against this blatant attempt at intimidation and the dismissal is a victory for all victims,” Troiani said in a statement. The Cosby lawsuit alleged that American Media Inc. continued to publish numerous stories about Constand’s claims regarding Cosby ”despite its clear contractual obligations to not publish any stories relating to certain allegations against Mr. Cosby.”  Additionally, Cosby claimed Constand gave an interview to the National Enquirer, which was published in January 2016    another alleged violation of the settlement. Dylan Howard, vice president and chief content officer at American Media, Inc. said ”we were always confident that AMI had not breached any agreement with Mr. Cosby, and we are pleased that he has decided not to pursue the litigation further.” More than 50 women have accused Cosby of sexual misconduct but Costand’s accusations are the first to result in charges being filed. The Cosby trial: How we got here',
 ' (CNN) The police may have found the missing dress. You know, the white custom Calvin Klein gown that adorned actress Lupita Nyong’o when she attended Sunday’s Academy Awards.  The one encrusted with 6, 000 white Akoya pearls that Fortune estimated was worth $150, 000. The dress that Nyong’o, who won a best supporting actress Oscar last year for ”Twelve Years a Slave,” reported stolen Wednesday from her room in the swanky London Hotel off Sunset Boulevard in Hollywood. Yes, that old thing. The L. A. Sheriff’s Department got a tip Friday and looked in an abandoned bathroom on the second floor of the same London Hotel, the department said in a press release. The tip, they said, came from ”a media representative” who got a call from an anonymous caller. The investigators went to the hotel and found a black garment bag underneath the bathroom counter.  Inside the bag they found a white dress ”resembling the one” worn by Nyong’o. Sounds like it would be hard to mistake the dress, but the sheriff’s department is working with the dress owners to find out if it’s the real thing.  Calvin Klein also expressed guarded enthusiasm over the dress’s potential return. ”All of us at Calvin Klein are thrilled to know that the dress has potentially been located. This was an amazing collaboration between the brand and Lupita and the dress looked truly exquisite on her,” Francisco Costa, Women’s Creative Director of Calvin Klein Collection, said in a press release. ”Once it’s returned to us, we will be able to have the dress restored and archived, as it now represents an important moment for the brand.” ',
 'Washington (CNN) Another woman stepped forward Saturday to accuse Donald Trump of sexual assault, saying he kissed her without her consent at his estate in Florida. Cathy Heller joins a rapidly growing list of women who are accusing the GOP nominee of sexual assault in the week since a 2005 tape surfaced in which Trump bragged about being able to grope women and get away with it. Heller, now 63, said she met Trump at    nearly 20 years ago while she and her family were having brunch, she told The Guardian newspaper. Heller said she had been introduced to the GOP nominee by her   . She told the paper the real estate magnate ”took my hand, and grabbed me, and went for the lips,” without her consent.   She said Trump held her in place and kissed her. CNN has not been able to independently confirm Heller’s claims. A spokesman for Trump’s campaign said there was ”no way” Heller’s account is true. ”The media has gone too far in making this false accusation,” Jason Miller said in a statement. ”There is no way that something like this would have happened in a public place on Mother’s Day at Mr. Trump’s resort. It would have been the talk of Palm Beach for the past two decades. The reality is this: for the media to wheel out a politically motivated Democratic activist with a legal dispute against this same resort owned by Mr. Trump does a disservice to the public, and anyone covering this story should be embarrassed for elevating this bogus claim.” Heller has donated to Democratic campaigns on multiple occasions, including a $2, 700 donation she made to Clinton’s presidential campaign this year. She also told The Guardian ”her husband’s family is involved in a   effort to recover initiation fees that her late   paid to join   .” But The Guardian said it corroborated Heller’s claims with a relative who was seated at the dining table as well as a friend whom Heller told a year and half ago. Heller is one of nearly 10 other women who have publicly accused Trump of sexual assault, including accounts of kissing and groping. Trump has vehemently denied all the allegations, suggesting they are surfacing now because of a ”smear” campaign to derail his candidacy. ”Remember this, it’s a rigged election because you have phony people coming up with phony allegations with no witnesses whatsoever,” Trump said at a campaign rally in Portsmouth, New Hampshire. ”The election is being rigged by corrupt media pushing completely false allegations and outright lies in an effort to elect her (Clinton) president.” CORRECTION: A previous version of this story misstated Heller’s version of events as reported by The Guardian.',
 '’Wealthy shoppers are reining in their spending because of  according to Restoration Hardware CEO .’ ’”’ ’He continued: ”Historically, our business has a correlation to large movements in stock prices as we believe asset valuations influence our customers’’ buying patterns. ”’ ’Even heavy discounts, such as 20% to 75% off everything in the store, have been failing to boost sales as much as in previous years.’ ’”O’ ’’ ’’ ’’ ’’ ’’ ’Baird analysts wrote in a recent note:’ ’Stifel analysts said that the strategy is innovative, but ”we think there aren’’t so many households that see themselves as wanting interior design all the time. ”’ ”Restoration Hardware’s shares hit a   low of $36. 42 in midday trading on Wednesday.” ’Wealthy shoppers are reining in their spending. ..’',
 '’  ’ ’   Larry King interviewed Republican presidential candidate Donald   Trump on his show, ”PoliticKING,” Thursday night, which  (RT). ’ ’   The two . ’ ’   After the   interview,   Trump for praising Russian President Vladimir   Putin and critiquing American media on the   news   organization. ’ ’   But the Trump campaign is saying it did not know the interview   would appear on RT. ’ ’   ”Mr. Trump recorded a short interview with Larry King for his   podcast as a favor to Mr. King,” the campaign said in a statement   to Business Insider. ”What Larry King does with the interview   content is up to him, we have nothing to do with it.” ’ ’   Similarly, Trump campaign manager  that ”nobody said it would be on Russian   TV.” ’ ”   Some    that the campaign didn’t know Trump’s interview would   appear on RT. ” ’   King’’s production company, Ora Media, sent Business Insider the   following statement: ”RT America is one of several dozen   companies who license [PoliticKING] and other Ora programs for   distribution around the world. A visit to www. Ora. tv will provide   several examples of our content.” ’ ’ ’ ’ ’t’t’tRegarding   interview with . ..  ’',
 '’  ’ ’   The   backed coffee chain that Silicon Valley is   obsessed with is expanding to three new cities this fall. ’ ’  ’ ’   Blue Bottle, which operates  globally,   mostly in California and New York, has become hugely popular with   hipsters and tech workers since it was founded in the early   2000s. Baristas craft one cup at a time using vintage brewing   machines and artisanal blends. ’ ’   The going rate for a cup of   coffee is about $3. ’ ’   The specialty coffee roaster has raised some $120 million in   venture capital, from   ’ ’  ’ ’   Fans herald Blue Bottle as the ”” where customers   are treated as individuals and the coffee provides a    ’ ’  ’ ’  ’ ’   ’',
 '’  ’ ”   Conservative author Ann Coulter, an outspoken Donald Trump   supporter, doesn’t like any of the names being floated as the   Manhattan billionaire’s possible running mate. ” ’   Former House Speaker Newt Gingrich and Govs. Mike Pence of   Indiana and Chris Christie of New Jersey are all bad ideas,   Coulter told Business Insider on Tuesday. ’ ’   She presented her theory on what she believes is really   happening. ’ ’   ”I suspected it with Newt, and when he started talking about   Pence I became convinced he’’s just throwing out absurd names so   that when he finally presents his vice president we’’ll all be so   relieved,” she said. ”’’Oh, at least it’’s not Newt  —   at least it’’s   not Pence.” ’ ”   Her choice for the role? Kansas Secretary of State Kris Kobach,   an early Trump supporter  of the Manhattan billionaire’s plan to build   a massive wall along the   border. ” ’’ ’   ”He’’s no Sarah Palin  —   he’’ll open his mouth and they’’ll find out   he’’s not Sarah Palin or Dan Quayle,” Coulter said, referring to   John McCain’’s running mate in 2008 and George H. W. Bush’’s vice   president. ”His issue is immigration. He’’s tall, he’’s young, he’’s    . But he could just be head of homeland security. I’’d   be OK with that.” ’ ”   Coulter’s second choice was Scott Brown, the former Massachusetts   senator who became the first person who served in the Senate to   publicly endorse Trump. ” ’   She said pitched the ideas ”to everyone I know who knows Trump,   who can get to Trump, and to Trump himself.” ’ ’   ”Oh I’’ve done everything,” she said. ”I’’ve sent carrier pigeons.” ’ ”   Trump’s decision is considered to be imminent. He met with Pence   in Indiana alongside his family Wednesday and was set to meet   later with Gingrich. ”',
 '’  ’ ”   Snapchat has been secretly hard at work on what has been   described as a competitor to Google Glass. A source has tipped   Business Insider off, however, that Snapchat’s supposedly     project may have been hiding in plain sight. ” ’   In August, Snapchat CEO Evan Spiegel was pictured wearing a very   different pair of sunglasses from his usual ones while on   vacation in Corsica with his girlfriend, the model Miranda Kerr. ’ ’   The paparazzi photos taken of Spiegel show him looking straight   at a camera with a very circular pair of frames and what appears   to be cameras on either side of them. ’ ”   Compared with Spiegel’s normal aviator style, the sunglasses   photographed in August appear to have thicker frames and the   attached cameras. They’re similar to but aren’t exactly clones of   the glasses made by Vergence Labs, a company Snapchat acquired in   2014. ” ’   The source told Business Insider of being ”shocked” that Spiegel   had apparently worn the early prototype on vacations and that   nobody noticed, though the CEO might have not been expecting to   be photographed at the secluded beach. The photo was also taken   months before news of Snapchat’’s hardware efforts surfaced, which   may explain why it went unnoticed at the time. ’ ’  ’ ’   Snapchat is ”very serious” about a hardware play, a different   source with knowledge of the situation told Business Insider, and   the company has been on a hiring spree to build up its team for   it. The company is regularly sending teams to China to work with   contract manufacturers for the project, according to another   source. ’ ”   Snapchat’s hardware ambitions first came to light in March,    to an earlier string of notable hardware   hires, including eyewear designer Lauryn Morries and a former   employee who listed his experience as working on highly   miniaturized and very   electronics. ” ’  ’ ’   CNET speculated that Snapchat could be working on a Google     device, which would mark a major expansion for the     startup, which is valued . ’ ’   The photos of Spiegel wearing the alleged early prototype   appeared six months before the report, meaning the company could   be further along in development than previously thought. ’ ”   Yet it’s still ramping up production. In the past six months, the   company has brought on two more hardware specialists from GoPro   and a handful of specialists in   electronics. Snapchat   also hired the technical recruiter for Google’s Project Aura, the   group responsible for Glass, and the technical recruiter for   Microsoft’s HoloLens project. ” ”   Snapchat didn’t immediately respond to request for comment. ” ” Do you know more about Snapchat’s plans? Email the reporter   at bcarson@businessinsider. com. ” ’Snapchat has been secretly hard at work on what. ..’',
 '        For us to continue writing great stories, we need to display ads.             Please select the extension that is blocking ads.     Please follow the steps below, NEWS BRIEF The Italian coast guard rescued more than 6, 000 migrants from the Mediterranean Sea, officials said Monday, in one of the largest   rescues of the European migration crisis. Nine migrants were found dead during rescue operations off the coast of Libya, including a    pregnant woman. Smugglers carried migrants on 40 rubber dinghies and at least two rafts. In one of Monday’s operations, members of the Italian coast guard rescued 725 migrants who were fitted on one boat. The Associated Press adds: The coast guard said it coordinated a total of 39 rescues Monday in the sea about 30 miles north of Tripoli. Smugglers took advantage of the first day of calm seas after days of rough waters to launch vessels crowded with migrants who pay them in hopes of reaching European shores. The rescue effort took around 10 ships to complete, Reuters reports. This comes just one week after a boat carrying as many as 600 migrants capsized off the Egyptian coast as it headed to Italy, where at least 160 people died. So far this year, 132, 000 migrants have arrived in Italy, while more than 3, 000 have died attempting to cross the Mediterranean, according to the International Organization for Migration.',
 '           For us to continue writing great stories, we need to display ads.             Please select the extension that is blocking ads.     Please follow the steps below, NEWS BRIEF Using remote sensing technologies, U. S. archaeologists have unlocked a lost piece of early North American history —  all without actually digging. The fort of San Marcos, located in   Parris Island, South Carolina, was one of five forts that existed in 1577 in the Spanish colonial town of Santa Elena, the remains of which were first uncovered almost 40 years ago. After two years of research, Chester DePratter of the University of South Carolina and Victor Thompson of the University of Georgia were able to uncover the missing fort by employing   radar, soil testing, and monitoring magnetic fields to detect the landscape of the ancient settlement. The   fort was part of one of the oldest Spanish settlements in the Americas, built by Spanish military officer Pedro Menedez Marquez in order to curb French expansion in the New World. Marquez constructed the fort in six days to defend the settlement against potential attacks from Native Americans. Though there were documentary sources proving the fort’s existence, previous attempts to locate it through archaeological excavation were unsuccessful. “I have been looking for San Marcos since 1993, and new techniques and technologies allowed for a fresh search,” DePratter said in a press release Monday. “Pedro Menendez didn’t leave us with a map of Santa Elena, so remote sensing is allowing us to create a town plan that will be important to interpreting what happened here 450 years ago and for planning future research. ” The archaeologists say the discovery, which currently sits beneath a former military golf course, will allow them to better understand the land’s history, and the European powers’ competitive expansion that helped shape it.',
 '           For us to continue writing great stories, we need to display ads.             Please select the extension that is blocking ads.     Please follow the steps below, NEWS BRIEF Wells Fargo has agreed to paid more than $4 million in a settlement after federal investigators found the bank illegally repossessed 413 cars owned by American servicemembers over the last seven years. The Justice Department announced the agreement Thursday, alleging Wells Fargo violated the Servicemembers Civil Relief Act, which gives certain legal protections to members of the military. The settlement comes after a yearlong investigation. According to investigators: Wells Fargo had repossessed Army National Guardsman Dennis Singleton’s used car in Hendersonville, North Carolina, while he was preparing to deploy to Afghanistan to fight in Operation Enduring Freedom.  After Wells Fargo repossessed the car, it sold it at a public auction and then tried to collect a deficiency balance of over $10, 000 from Singleton and his family. Investigators found that Wells Fargo failed to obtain necessary court orders to repossess Singleton’s car, along with more than 400 others. A federal court must now approve the settlement, which also restores credit to the servicemembers affected. Wells Fargo was fined additionally for the scheme, CNN reports: The bank was fined $20 million more by the Office of the Comptroller of the Currency for breaking three provisions of the same law by denying members of the military certain banking protections, including capping their interest rates at 6%. Those violations began in 2006, the OCC said. The bank is already under heavy scrutiny for its fraudulent practice of creating fake bank and   accounts to collect fees. CEO John Stumpf was lambasted by members of the House Financial Services Committee earlier Thursday in his second visit to Capitol Hill in as many weeks. Committee Chairman Jeb Hensarling said: Mr. Stumpf, I have a mortgage with your bank. I wish I didn’t. I wish I was in the position to pay it off because you have broken my trust as you have broken the trust of millions. Several lawmakers demanded Stumpf step down as CEO. One Democrat even suggested the need to break up the bank, saying it is too large to control.',
 'London Attack: What We Know, The claim, which has been published in Arabic, German, and French, was not unlike previous statements the group has used to take responsibility for attacks. When it took credit for the Bastille Day attack in Nice last July, in which 86 people were killed, and the truck crash at a Berlin Christmas Market last December, in which 12 people were killed, the group offered similar wording by calling the attacker an ISIS “soldier” who targeted “citizens of the Crusader coalition. ” In both cases, the group waited at least one day before claiming the attack and refrained from identifying the attacker by name, making it unclear if the group was aware of the individual’s identity before the attack took place. Such behavior is typical of the Islamic State. As New York Times reporter Rukmini Callimachi noted Thursday, ISIS’s claim of responsibility followed its usual script and timeline. The group has, with few exceptions, consistently only claimed attacks it could prove it directed (as was the case of the Berlin market attack, for which it released a video of the attacker pledging allegiance to ISIS) or suggest it inspired (as in the case of Nice). Though the details surrounding the London attack are still emerging, ISIS has not yet released a video of Masood pledging allegiance, suggesting he may not have been directly commissioned. But where there are similarities, there are also notable differences. As my colleague Uri Friedman noted, the attack in London targeted the highly guarded Westminster Palace, marking a departure from previous attacks, which have usually targeted   areas. Also unlike past attacks, which the group has actively promoted, the assault in London was conspicuously absent from the latest issue of ISIS’s   newspaper, further suggesting the attack may not have been anticipated. While some have suggested ISIS’s nearly   delay in claiming responsibility raises doubt over how directly it was actually involved in the London attack, versus simply claiming credit for an individual’s act of violence after the fact, the lull in information may have been intentional. As Charlie Winter noted in The Atlantic after last year’s attack in Brussels, claiming responsibility on its own time allowed ISIS to provoke speculation about the attack and, ultimately, command the media narrative surrounding it. It’s still early, and more concrete evidence tying ISIS to the London attack could emerge later. But for now, observers are left with only the information released by authorities and ISIS’s word. And the uncertainty itself works in the group’s favor.',
 ', I want to receive updates from partners and sponsors. The United Kingdom said Wednesday it will take in 3, 000 child refugees from the Middle East and North Africa, a new program that adds to its existing plan to accept 20, 000 Syrian refugees by 2020. The U. K.’s Home Office called it the “largest resettlement effort” in the world to help children from these areas, adding it would work with the United Nations High Commission for Refugees to resettle the children. The U. K. Home Office said: On the UNHCR’s recommendation, the scheme will not solely target unaccompanied children, but will also extend to vulnerable children at risk, such as those threatened with child labour, child marriage and other forms of abuse or exploitation. It will be open to all ‘at risk’ groups and nationalities within the region. The U. K. has already resettled some 1, 000 Syrian refugees, more than 500 of whom are children. However groundbreaking, the announcement wasn’t enough to satisfy the Liberal Democratic Party leader Tim Farron. He called it a “cynical ploy” because it would not help migrants already in European camps. He told the BBC that of the 30, 000 unaccompanied migrant children already in Europe, some 10, 000 had gone missing, and are feared to have been taken by human traffickers. Farron said Prime Minister David Cameron could have done much more, “and yet once again he’s failed to take the opportunity. ”             Like this? Subscribe to the Atlantic Daily, a newsletter with stories,             ideas, and images from The Atlantic.        ',
 ' Newly unearthed photos of President Obama in Muslim garb underscore his deep ties to the faith    and possibly help explain his reluctance to call out radical Islam, Fox News Channel’s Bill O’Reilly said Wednesday. The photos, aired on “The O’Reilly Factor,” were taken at the wedding of Obama’s   Malik Obama, in Maryland sometime in the early 1990s, O’Reilly said. They were not offered as evidence that Obama is Muslim, as some of his critics insist, but to show his “deep emotional ties” to the religion of his father and stepfather, O’Reilly said. “There is no question the Obama administration’s greatest failure is allowing the Islamic terror group ISIS to run wild, murdering thousands of innocent people all over the world, including many Muslims,” said O’Reilly. “Mr. Obama has never, never acknowledged that mistake, nor does he define the ISIS threat accurately. “That group is killing innocent people in order to impose a radical version of Islam on the world,” the newsman added. “The jihad is solely based on theology, perverted as it may be. ” A similar photo emerged in 2008, when Obama was running for the Democratic presidential nomination against Hillary Clinton. In that photo, Obama was seen wearing a white turban and Muslim dress. The photo was sent to the influential Drudge Report, and Obama’s campaign blamed Clinton for the “smear. ” ”On the very day that Senator Clinton is giving a speech about restoring respect for America in the world, her campaign has engaged in the most shameful, offensive   we’ve seen from either party in this election,”   campaign manager David Plouffe said. The president reportedly met his older   for the first time in 1985. Malik Obama now lives in a rural Kenyan village near Lake Victoria.   ',
 ' Facebook activated its Safety Check service for the first time in the U. S. after Sunday morning’s deadly shooting in a packed Orlando nightclub that killed 50 people and wounded at least 53 more. ISIS claimed responsibility for the attack Sunday afternoon via its Amaq news agency, according to Reuters. Designed for times of disaster or crisis, Safety Check lets users in a specific area notify friends and family that they are safe. Users can also check on others in the affected area and mark friends as safe. Related: Belgians harness social media to help people affected by Brussels terror attacks, The social network activated its Safety Check feature following the Brussels terror attacks and the bombing in Ankara, Turkey earlier this year, as well as the attacks that rocked Paris last year.  Earlier this year Facebook apologized for a glitch in its Safety Check service that sent out misdirected notifications after the Easter Sunday bombing that killed 70 people in Lahore, Pakistan. The shooting at Orlando gay club Pulse dominated social media Sunday, with #Orlando the top trending U. S. topic on Twitter. News coverage of the attack also appeared in Facebook’s Trending Topics section. Follow James Rogers on Twitter @jamesjrogers, ',
 ' For the first time since 2012, over half of voters approve of the job Barack Obama is doing as president.  At the same time, more voters than not think his administration has made the country weaker, and views are mixed over whether the president’s policies have helped the economy.  These are some of the findings from the latest Fox News poll.  The poll was conducted before Sunday’s mass shooting in Orlando. While 37 percent of voters think Obama administration policies have helped the economy, nearly as many    35 percent    say they’ve hurt.  Another 27 percent say they haven’t made much difference.  CLICK TO READ THE POLL RESULTS, In addition, by a   margin (  percent) more think the country is weaker under Obama’s leadership.  That’s a significant improvement from two years ago, when it was a   margin:  55 percent weaker vs. 35 percent stronger (June   2014). The latest headlines on the 2016 elections from the biggest name in politics. See Latest Coverage → Meanwhile, 51 percent of voters approve of Obama’s job performance and 46 percent disapprove.  Last month it was   percent.  The last time Obama’s approval rating was at 50 percent or better was late October 2012.  In that survey, conducted right before he won   51 percent approved and 46 percent disapproved (October   2012). Most Republicans (86 percent) think the administration has made America weaker, as do 49 percent of independents and 17 percent of Democrats.  Two years ago, 89 percent of Republicans, 59 percent of independents and 22 percent of Democrats felt that way. Those most likely to feel Obama’s policies have helped the economy include blacks (68 percent) Democrats (65 percent) voters under age 30 (48 percent) Westerners (44 percent) and those in households earning less than $30, 000 annually (41 percent). The Fox News poll is based on landline and cellphone interviews with 1, 004 randomly chosen registered voters nationwide and was conducted under the joint direction of Anderson Robbins Research (D) and Shaw  Company Research (R) from June   2016.  The poll has a margin of sampling error of plus or minus three percentage points for all registered voters.   ',
 '    The U. S. Coast Guard is searching for a cruise ship passenger who may have gone overboard in the Gulf of Mexico Friday. The Coast Guard said in a statement that its district command center in New Orleans was contacted by the master of the cruise ship Carnival Liberty. The ship reported that surveillance video showed a woman falling overboard at about 2 a. m. Friday. “Earlier today, a    female guest on the cruise ship Carnival Liberty was reported missing by traveling companions. A   search was conducted along with a review of the ship’s camera footage. A   search was conducted along with a review of the ship’s camera footage,” the cruise line said in a statement, according to  . DFW: NTX woman missing after falling off cruise ship https:  .  “Unfortunately, it appears the guest may have gone overboard early this morning. Authorities were notified including the U. S. Coast Guard which is initiating a search effort in the area where the ship  was located when the individual was last seen. Carnival’s CareTeam is providing support to the guest’s traveling companions and family. We are keeping our guest and her loved ones in our thoughts during this difficult time. ” Samantha Broberg, 33, was found to be missing. A Coast Guard aircraft was dispatched from Mobile, Alabama, to search the area about 200 miles southeast of Galveston. The ship had left Galveston for Cozumel, Mexico about 10 hours earlier. The Associated Press contributed to this report.',
 '“We had been there maybe 15 minutes, standing in ankle deep water and all of a sudden I looked to my left and within two feet I saw this shark’s tooth, which I thought it was a shark’s tooth but I’ve never seen anything that large,” Campbell said. “My wife though it was a rock. And I reached over and picked it up and I saw the serrated edges so I knew then that it was a big shark’s tooth, but I had no idea it was a Megalodon. ” Campbell and his wife took the tooth to Ripley’s Aquarium in South Carolina and spoke with Brady Stoever, an aquarist who told them it was no ordinary tooth it was likely “millions of years old. ”  Turns out, it came from what used to be the world’s largest shark. According to Stoever, the tooth came from a Megalodon, a shark that lived millions of years ago and could grow up to 60 feet in length a great white only grows to be no more than 20 feet. Campbell says he is keeping the tooth: “My wife is talking about getting a shadow box and put it in it and just keeping it. .. it is a great conversation piece.”',
 'The betting markets seem to be coming around to the realization that it is highly unlikely that anyone but Donald Trump or Ted Cruz will be the Republican nominee. Not that betting markets are anything but a brass tacks version of conventional wisdom. But the perception is correct. We seem to be in a period where Republican party stakeholders are taking an inordinate amount of glee in pointing out that as long as Donald Trump doesn’t clearly win more than 50% of the pledged delegates they can really do anything they want in Cleveland. This is true. But the logic is like getting out of putting up your relatives by burning your house down. Perhaps the greatest irony of the entire election cycle is that Republicans seem to have managed what a month or more ago seemed all but impossible: blocking Trump from securing the nomination outright. But that came at the expense of empowering Ted Cruz. He became their weapon of choice, the only feasible tool to deny Trump delegates, not only at the ballot box but just as much in the battle for ’dark delegates’. In some ways the more interesting battle now is not Trump vs the Republican establishment but the Republican establishment vs Cruz as they look for a way to get him to cough up the delegates they have helped and allowed him to garner. Josh Marshall is editor and publisher of TalkingPointsMemo. com. ',
 'Two Northwestern University students were charged with a hate crime after they allegedly   racist, homophobic messages and Republican presidential candidate Donald Trump’s name inside a nondenominational chapel on campus, The Chicago Tribune reported Saturday. Anthony Morales, 19, left, and Matthew Kafker, 18, were charged with institutional vandalism, hate crime to a place of worship, and criminal damage to property, according to the Tribune. A Cook County judge ordered they be held in lieu of a $50, 000 bail for allegedly painting the messages at the Alice Millar Chapel earlier in the week. The pair allegedly painted over a photo of Muslim students in the chapel, in addition to painting a swastika, expletive and slur against African Americans, a derogatory word for homosexuals and Trump’s name, according to the report.',
 'After decisively winning the Iowa GOP caucuses late Monday night, Sen. Ted Cruz ( ) asked Democrats to join him in the race to the White House in an emotional,   speech that ran more than 30 minutes. ”Tonight, I want to say to every member of the Democratic Party who believes in limited government, in personal opportunity, in the United States Constitution and a safe and secure America, come home,” Cruz said. ”To the Reagan Democrats, your party has left you. The Republican Party wants you we welcome you back.” Joined on stage by his wife Heidi Cruz, his father Rafael Cruz, and his senior Iowa campaign staffers, the Texas senator gave a lengthy speech that leaned heavily on his spiritual beliefs and his political battles against the “Washington cartel. ” The lengthy   prompted all four major cable networks —  CNN, Fox News,   and MSNBC —  to cut away to the Democratic candidates before the Iowa victor was finished speaking.  Cruz applauded his supporters for casting the most votes ever recorded in any Republican primary —  a statistic supported, at least in the Iowa caucus, by the Des Moines Register’s totals for all the presidential elections since 1976. According to the Associated Press, Cruz received 27. 7 percent of the vote, with 51, 047 votes counted for him as of 12 a. m. ET.  The Texas senator framed his Iowa win as a crucial step in his crusade to restore “the   values that built this great nation. ” “Our rights come from our creator,” Cruz said. “And the federal government’s role, the federal government’s responsibility is to defend those fundamental rights, to defend us. ” “While Americans will continue to suffer under a president who has set an agenda who is causing millions to hurt across this country I want to remind you of the promise of scripture,” he said later in the address. “Weeping may endure for a night, but joy cometh in the morning. Iowa has proclaimed to the world, morning is coming. Morning is coming. ”',
 'CHICAGO (AP)  —   A professor of a Christian college who asserted Christians and Muslims worship the same God will leave her post, the school near Chicago announced. In a statement posted on Wheaton College’s website, the school and Larycia Hawkins said they ”have reached a confidential agreement under which they will part ways.” The statement didn’t go into details. The college had placed Hawkins, a Christian and an associate professor of political science, on leave in December after she posted her views on Facebook. Hawkins also began wearing a hijab, the headscarf worn by some Muslim women, to counter what she called ”vitriolic” rhetoric against Muslims. Shortly afterward, the college began efforts to dismiss her despite objections by some faculty members. ”I appreciate and have great respect for the Christian liberal arts and the ways that Wheaton College exudes that in its mission, programs, and in the caliber of its employees and students,” Hawkins said in the statement posted Saturday on the school’s website. College officials and Hawkins said in the statement that neither will speak publically about their relationship before a news conference scheduled for Wednesday. ”Wheaton College sincerely appreciates Dr. Hawkins’ contributions to this institution over the last nine years,” President Philip Graham Ryken said in the statement. ”We are grateful for her passionate teaching, scholarship, community service and mentorship of our students.” College officials previously had said Hawkins’ views were inconsistent with the college’s ”doctrinal convictions.” Hawkins had maintained that nothing about her views contradict the college’s statement of faith, which all instructors sign. Among other things it affirms belief in ”one sovereign God,” views on Adam and Eve and the existence of Satan. Nearly 3, 000 undergraduate and graduate students attend Wheaton College, which is roughly 30 miles from Chicago. Copyright 2016 The Associated Press. All rights reserved. This material may not be published, broadcast, rewritten or redistributed.',
 'Carly Fiorina on Wednesday urged the Republican National Committee to convince ABC News to let her into its presidential debate Saturday in New Hampshire. The network had previously announced that it would not hold an undercard debate, effectively giving the former   CEO, Sen. Rand Paul ( ) former Pennsylvania Rick Santorum, and former Arkansas Gov. Mike Huckabee the boot. All of those candidates, with the exception of Fiorina, dropped out of the race after the Iowa caucuses. Fiorina, the only woman in the GOP field, wrote in an open letter she posted to Medium that networks like ABC were making up debate rules ”arbitrarily.”  ”Now it is time for the RNC to act in the best interest of the Party that it represents,” she wrote. Fiorina noted that by earning 1. 86 percent of the votes in the Iowa caucuses, she beat New Jersey Gov. Chris Christie (1. 76 percent) and Ohio Gov. John Kasich (who tied Fiorina with 1. 86 percent of votes, but earned 11 fewer votes than the former HP executive). ”There are only 8 candidates left. It’s time for the RNC to demand that media executives step aside and let voters hear from all of us,” Fiorina wrote. ”I trust you will act appropriately.” Spokespersons for ABC News and the RNC didn’t immediately return TPM’s requests for comment.',
 ' Abe Vigoda, the character actor who found fame in his senior years for his roles in The Godfather and the television comedy Barney Miller, died Tuesday at age 94. His death was confirmed by his daughter, Carol Vigoda Fuchs, to the Associated Press. Fuchs said her father died in his sleep on Tuesday morning at her home in Woodland Park, New Jersey, of “old age. ” “This man was never sick,” she said. Born in Brooklyn, New York, in 1921, Vigoda began acting professionally in 1947, making a career of supporting parts in theater and television, including   appearances in the gothic soap Dark Shadows, which became a cult hit. But Vigoda’s breakout role came in his villainous turn in Francis Ford Coppola’s 1972 mafia film, The Godfather. Playing the part of Sal Tessio, Vigoda’s character plotted to kill Michael Corleone (Al Pacino) after the death of his father Vito (Marlon Brando) in a bid to take over the crime family. However, Tessio’s plans are ultimately foiled by Corleone, who instead has his rival executed. “I found him to be a quiet man. He kept mostly to himself,” Vigoda recalled to CNN in a 2008 interview of working with Brando. “I respected his privacy. He was a great star. This was my first big thing  —   I felt lucky and grateful I was with these people. ” He went on to appear in flashback scenes in the film’s sequel, The Godfather: Part II in 1974. Between   Vigoda appeared alongside Hal Linden on the T. V. show Barney Miller as crotchety Detective Phil Fish, a role he played in a   show, Fish, between  . “There’s something about my countenance, my demeanor if you wish, that suggests I have the world on my back,” he told the Associated Press in 1977 of his sudden fame for playing gruff characters. With a face, if not a name, well known to many Americans, Vigoda’s lengthy career and old age became a   joke among many in the entertainment industry. The actor often appeared on late night shows hosted by David Letterman and Conan O’Brien to poke fun at his old age or incorrect reports of his death. Unlike bigger stars, Vigoda often remarked that he enjoyed being recognized by fans on the street. “It’s a very nice feeling that people recognize you,” he told the AP, “particularly when it happens in middle age and you’ve been at your field for so many years and nobody knew you. ” His appreciation for his fans, he said, came from a sense of earnest gratitude that he was able to make a career pursuing his passion of acting. “So it does prove to me something: That if you don’t make it in your youth, you can make it in your middle age or even old age,” he said. “You believe in yourself, you should pursue it to the end of time, as long as you’re on this earth. ”  ',
 ' Following 2016’s unprecedented bleaching event in the northern third of the reef, bleaching has now struck in the reef’s middle section, meaning   of the world’s largest living structure have been affected. The bleaching, which is caused when high water temperatures force corals to expel photosynthetic algae called zooxanthellae, turning the coral white, now affects a 1, 500km ( ) stretch of the reef. “The bleaching is caused by   temperatures driven by global warming. This year, 2017, we are seeing mass bleaching, even without the assistance of El Niño conditions,” said Professor Terry Hughes from James Cook University’s Centre of Excellence for Coral Reef Studies. “Clearly the reef is struggling with multiple impacts,” explained Hughes. “Without a doubt the most pressing of these is global warming. As temperatures continue to rise the corals will experience more and more of these events: 1°C of warming so far has already caused four events in the past 19 years. ” Dr James Kerry, who conducted the aerial surveys with Hughes, warned that as bleaching events occur in    years, recovery will be almost impossible in some regions. “It takes at least a decade for a full recovery of even the   corals, so mass bleaching events 12 months apart offers zero prospect of recovery for reefs that were damaged in 2016,” he said. This year’s bleaching event may be exacerbated by tropical cyclone Debbie, which struck Queensland in late March. An assessment of the damage caused by the category four cyclone is currently underway, but scientists fear it may have caused significant damage to corals that were just beginning to recover from last year’s bleaching episode. Scientists have been highly critical of the federal and Queensland state governments’ support for the proposed Adani Carmichael coal mine in Queensland’s Galilee Basin. If it’s built, the mine will be the largest in Australia, producing 60 million tonnes of coal a year and significantly contributing to global greenhouse gas emissions. A decision on the mine’s future will be made in the coming months. “Ultimately, we need to cut carbon emissions, and the window to do so is rapidly closing,” Hughes warned.  ',
 ' On last night’s Real Time with Bill Maher, a talkshow that airs weekly on HBO, the guests included Piers Morgan and Australian comedian Jim Jefferies. Towards the end of the programme, Maher asked whether people who said during the US election campaign that Hillary Clinton was the “lesser of two evils” should apologise, in light of president Donald Trump’s executive orders to build a wall between the US and Mexico, and implement a travel ban on people from seven   countries. As the clip below shows, that’s when it all kicked off. When Morgan questioned why people should apologise, Jefferies described Trump’s travel restrictions as a “Muslim ban” to which Morgan said “there is no Muslim ban”. At this point Jefferies interjected to say: “Oh, fuck off. This is what you do, Piers  —  you say, ‘he hasn’t done this, he hasn’t done that, he’s not going to do all these things’. Give him a fucking chance mate  —   Hitler didn’t kill the Jews on the first day, he worked up to it. ” Jefferies added: “You just like that you won The Apprentice and you have a famous friend, mate. ” Yes, watching Piers Morgan being told to fuck off on live TV is *exactly* as satisfying as I’d always imagined. https:  .  Morgan described this as “ridiculous, hysterical    nonsense” and accused Jefferies of losing the audience, to which the comedian flipped him off. The former CNN host made light of the   row after the show. He said Jefferies had made himself look “hysterical and silly. Shame, as he’s usually quite funny. ” Fairly sure this is an Australian mating signal. @RealTimers, Morgan also defended saying Trump’s travel restrictions, currently halted by US judges, was not a Muslim ban, writing that “I don’t agree with the travel ban but it’s not a Muslim ban if 85% of the world’s Muslims aren’t affected. ” Jefferies meanwhile tweeted a video of the clip, asking Morgan if he wanted to retweet it.  ',
 '  Apple CEO Tim Cook broadcast an   memo to US Apple employees Wednesday evening calling for unity amid the uncertainty inspired by Donald Trump’s upset presidential win. In the memo, obtained by BuzzFeed News, Cook  —   in a nod to one of the most divisive presidential races in American history  —   tells Apple employees that “the only way to move forward is to move forward together. ” And he reasserts Apple’s commitment to social progress and equality. Cook does not mention Trump  —   who has publicly threatened Apple and its positions on data privacy and offshore manufacturing  —   by name. He doesn’t grant that Trump’s behavior during his presidential campaign was antithetical to Apple’s positions on tolerance, deiversity and inclusion. Nor does he acknowledge the disparity between Trump’s philosophies and the “This is about how we treat each other as human beings” sentiments Cook himself has expressed in   and interviews. Instead, he simply says: “Our company is open to all, and we celebrate the diversity of our team here in the United States and around the world  —   regardless of what they look like, where they come from, how they worship or who they love. ” In June, Apple refused to fund or provide support to the 2016 Republican National Convention due to the company’s opposition to Trump’s   nomination. You can read Tim Cook’s full memo below. Team, I’ve heard from many of you today about the presidential election. In a political contest where the candidates were so different and each received a similar number of popular votes, it’s inevitable that the aftermath leaves many of you with strong feelings. We have a very diverse team of employees, including supporters of each of the candidates. Regardless of which candidate each of us supported as individuals, the only way to move forward is to move forward together. I recall something Dr. Martin Luther King, Jr. said 50 years ago: “If you can’t fly, then run. If you can’t run, then walk. If you can’t walk, then crawl, but whatever you do you have to keep moving forward. ” This advice is timeless, and a reminder that we only do great work and improve the world by moving forward. While there is discussion today about uncertainties ahead, you can be confident that Apple’s North Star hasn’t changed. Our products connect people everywhere, and they provide the tools for our customers to do great things to improve their lives and the world at large. Our company is open to all, and we celebrate the diversity of our team here in the United States and around the world  —   regardless of what they look like, where they come from, how they worship or who they love. I’ve always looked at Apple as one big family and I encourage you to reach out to your   if they are feeling anxious. Let’s move forward  —   together! Best, Tim,   ',
 '’Following days of delays and confusion, Kanye West’s latest album The Life of Pablo finally dropped early Sunday morning. The album appeared on Tidal shortly after West’s performance on Saturday Night Live.’ ’Right now there are two ways to listen to the album. The first is by streaming it on Tidal.’ ’If you don’t want to sign up for Tidal, you can also just buy the album for $20 here. Once you put in your information, you’ll be able to download the album’s songs either as MP3 or FLAC files. ’] On Friday, West explained the delay, tweeting that it happened because Chance the Rapper wanted the song “Waves” on the album. If you don’t have a Tidal membership, you can sign up for a free trial here. Finally, Rolling Stone reported that The Life of Pablo will become available from other music retailers in one week. So if you don’t want to sign up for Tidal, hold tight for a few more days and hope there are no more delays.   ',
 'For months, Donald Trump has complained that he should not have to win a majority of delegates to the Republican convention to be the party’s nominee. We were among those who insisted to the contrary  —   that he had to win a majority and otherwise abide by the procedures the party had set forth in advance of the nomination contest. Now that he has won the Indiana primary and Senator Cruz and Governor Kasich have dropped out of the race, he is guaranteed to do that: to win fair and square, without the threat of violence in Cleveland that he had previously and shamefully raised. His victory demonstrated some real strengths that it would be foolish to deny. His mastery of the media was one, and we do not make that observation backhandedly: Would that a conservative of good character had displayed such an ability to use the networks to convey his messages. Trump had a better sense of where Republican voters are on immigration than most of the other candidates (even if he has taken no interest in the crucial details). His campaign has also shown boldness and imagination. Who else would have tried to win while spending almost no money? Who else would have ignored the strategists and consultants and just winged it, day after day, and successfully too? There ends our praise. We regret that Trump will be the Republican nominee and think Senator Cruz, our preferred candidate, would have been vastly better. Trump has done little to demonstrate any commitment to, or even understanding of, conservative principles his instinct seems to be to use government power to silence his critics he has no experience in government, a lack that we persist in seeing as a bad thing his ethical record is disturbing he will simply make things up when it suits his purposes he traffics in conspiracy theories about everything from Iraq to the JFK assassination he exhibits little  . We assume that in coming days we will hear even more discussion than previously of a new, more “presidential” Trump in the offing. We’ll believe it when we see it sustained. Trump has won more primary votes than any nominee before him but it is also true that no nominee has seen more primary votes cast for his opponents. He eked out a bare majority in Indiana at a time when past nominees were winning supermajorities. Any other nominee in this weak position would now turn to unifying his party. But Trump has in recent days said that he can win without doing that. If he finds a way to win the general election without nearly uniform support from Republicans, he will again have broken the mold of modern politics. He enters the race as an underdog against Hillary Clinton, who is, thanks entirely to him and notwithstanding her own primary defeat in the state, the other great victor in Indiana. ',
 'The upcoming Disney film Moana will star a Polynesian princess for the first time in the franchise’s history  —   but, rather than be excited, many people are complaining that the film’s portrayal of a Polynesian demigod character is culturally insensitive because he’s too fat. According to MauiNow. com, Polynesian politicians are among those speaking out. For example, New Zealand Labour MP Jenny Salesa (who is of Polynesian heritage) complained on Facebook that the movie having a fat demigod will actually harm children. “It is great that Moana is the lead,” she continued. “However, it is disappointing that Maui, one of our beloved historical ancestors from hundreds of years ago, who was a very strong man and a skilled navigator, is depicted to be so overweight in this kids’ movie. ” In an interview with MauiNow. com, a University of Hawaii graduate student Oriana Leao  —   who is working on a thesis titled “Misrepresentation of Kanaka Maoli Women in American Cinema”  —   said that she basically agrees that we should be offended. Here’s the thing: Aren’t people also constantly complaining that movie characters are not fat enough? In fact, just recently, an advocacy group called ProjectKnow declared that even cartoon superheroes should be fatter, because  —   apparently despite the fact that fighting crime, much like “scooping up entire islands” would probably get you into pretty good shape  —   not seeing enough fat cartoon superheroes is (you guessed it!) hurting children. Gee, it’s almost like there’s no way to win. Now, not everyone agrees with people like Salesa. For example, Samoan illustrator Michel Mulipola defended Disney’s decision in a   Facebook post on the grounds that the character’s size fits with his personality. “As an illustrator who works in an industry similar to animation, I understand the meaning of shapes and silhouettes when designing characters,” he wrote. “If you study animation, you’ll notice that each character has a distinctive ’shape’ to their design to not only distinguish who is who but also to portray personalities through their look. ” More P. C. Culture   Histrionics on the LeftFinding the Roots of Violent Campus ProtestsWhy the Liberal Elite Will Never Check Its PrivilegeAlong with the post, Mulipola included a graphic breaking down how Maui’s body type is consistent with what the body of a character with his personality and role would usually look like, regardless of race: “I’m not phased by the way Maui is designed in this film,” he writes. “In Polynesian mythology, Maui is the   who inspired many myths and legends. ” “In this film, he would of done a lot of those amazing feats but he’ll be a blow hard [sic] braggart who can be a bit of an idiot. ” Honestly, his explanation seems both logical and   . But who needs thinking when you can just get angry instead?  —   Katherine Timpf is a reporter for National Review Online.  ',
 'With its inherent contradictions and absurdities, war has always provided satirists with a   environment. Fighting men themselves have a keenly developed sense of black comedy and love to joke about futility, the opaque nature of military jargon, the obliviousness of their officers, and a thousand other bedevilments. \tBut the limp new Netflix satirical comedy War Machine, which casts Brad Pitt as a barely fictionalized version of our former top commander in Afghanistan, General Stanley McChrystal, is an exercise in shallow ignorance. To adequately satirize something, it’s preferable to have some understanding of it, and   David Michôd has none whatsoever. Instead he combines lazy, broad jokes  —   such as a running gag about how people keep addressing a guy named Richard as “Dick”  —   with an analysis of military history that is about as informed as that of the average college sophomore who owes his knowledge on the subject to whatever Vietnam movies he’s seen. \tMichôd, an Australian who made 2010’s Animal Kingdom, seems barely acquainted with even the basics of his subject, such as when military men salute or wear headgear, and all over the film there are strangely discordant touches, such as when McMahon makes a joke about turning off Fox News, even though the TV nearby is clearly showing CNN’s Jeffrey Toobin. The director inexplicably casts the millennial actor John Magaro as a full colonel though he is easily a decade too young for the part and makes Cullen, the ostensible voice of reason in the piece, sound like a preening, sneering ass by giving him dialogue like, “These guys thought they were the most important guys in the world . . . maybe they assumed I thought they were as amazing as they did. ” It’s not a great idea to portray a freelance magazine writer as dripping with condescension for men fighting the Taliban. War Machine lands a few jabs on President Obama, who appears in a clip from his West Point speech and is also portrayed in passing by Reggie Brown. McMahon is shown being dumbfounded when he sees Obama, on TV, simultaneously announcing that he is beefing up the armed forces in Afghanistan by 30, 000 troops and that he will start bringing them home in only 18 months, thus signaling to the bad guys his lack of seriousness. Later in the film, having met just once with McMahon, Obama summons the general to a meeting aboard Air Force One, then leaves him standing embarrassed on the tarmac after a quick handshake. More thinking along these lines would have expanded the film’s scope and perhaps nudged it closer to the tone of In the Loop (2009) a   Iraq War satire that was caustically funny (and led to the HBO series Veep). As it is, however, War Machine is a vacuous gesture of contempt that says nothing more interesting than “Can you believe how dumb and arrogant these military guys are? ”,',
 'A feminist student group at the University of Mary Washington has filed a Title IX lawsuit with the school, claiming that it failed “to protect them from a sexually hostile school environment” because it didn’t block a   app on which students were saying bad things.  \tThe app in question, Yik Yak, allows people to post anonymous messages that people in the same geographical area can see. According to an article in The College Fix, the school’s Feminist United group claimed that some students were using the app to post “overtly sexist   threatening” messages  —   and that the school’s failure to shut Yik Yak out of the school’s WiFi created a sexually hostile environment in violation of both Title IX and the Fourteenth Amendment.  \tNow, The Fix reports that the school is maintaining that the group has a lack of evidence to support its claim, but I’d like to take it a step further: Their claim is dumb.  \tNo, the school didn’t ban Yik Yak, but it also didn’t force these students to log on to it and look at the things that were upsetting them so much. The “yaks” that were bothering these students didn’t even qualify as threats, and even if they did, that still wouldn’t mean that the app the threats were posted on would be responsible  —   let alone that the school would be responsible for allowing that app. I receive death threats on Twitter on a pretty regular basis should I be suing places for letting me see Twitter using their WiFi? It’s not hard to see how stupid this is it’s much harder to see how anyone could actually take this seriously. I mean, you’re a student in college and this is what you’re doing with your time? Calm down and go drink a beer like a normal person.   —   Katherine Timpf is a National Review Online reporter.\t  ',
 'The divinity schools at Duke and Vanderbilt Universities have instructed their professors to start using more “inclusive” language when referring to God because the masculine pronouns “have served as a cornerstone of the patriarchy. ” For example: This year’s divinity course catalogue at Vanderbilt tells professors to give “consistent attention to the use of inclusive language, especially in relation to the Divine,” because the school “commits continuously and explicitly to include gender as an analyzed category and to mitigate sexism. ” Now, that may sound fair, but in many cases, it’s really not up to the professor. For example, if we are talking about the Christian God, every single reference to Him in the Bible uses a masculine pronoun . . . which kind of gives you the vibe that Christians have decided that their god is a dude. The fact is, teaching anything else would be giving inaccurate information  —   which is what makes Duke’s particular guidelines even more absurd. According to Heat Street, Duke’s particular divinity school is “geared toward people already working in the Methodist church, taking supplemental weekend or summer classes. ” Yes, “Methodist,” as in the Christian religion that has already completely, officially, 100 percent decided that their God is a man. And yet, Duke’s guidelines suggest avoiding gender specific pronouns when discussing Him and suggest using “God” and “Godself” instead. Look: The great thing about this country is that your religion can be whatever you want it to be. If, in your eyes, God is a woman or genderfluid or a microwave, then you can totally refer to God as being a woman or genderfluid or a microwave. Literally no one is stopping you. In fact, there is an entire Constitution protecting your right to worship His Holiness Microwave if that’s how you want to live your life. But if you are talking about the God of the Methodist religion, then it’s just plain inaccurate to refer to Him as anything but “Him. ” It would be like teaching Hamlet and calling Hamlet “she. ”  There is a point where an obsession over political correctness can blind people from basic of facts, and call me archaic, but I really do feel like facts are still the way to go.,',
 'New Yorkers better stock up! More than 1, 000 Big Apple bodegas are expected to shut down Thursday in protest of President Trump’s immigration ban. The strike by   store owners is set to take place from   p. m. across the five boroughs,  . “This shutdown of grocery stores and bodegas will be a public show of the vital role these grocers and their families play in New York’s economic and social fabric,” the page reads. “During this period, grocery store owners will spend time with their families and loved ones to support each other; many of these families have been directly affected by the Ban. ” Yemen is one of the seven   nations facing a temporary ban on entering the US as a result of Trump’s executive order. In addition to the protest, the bodega owners will also be holding a public call to prayer at sundown and a rally at Brooklyn Borough Hall at 5:15 p. m. “The program will begin with the Muslim call for prayer and a public sundown prayer by Muslim rally participants,” the Facebook page explains. “The prayer will be followed by several Yemeni merchants and their families sharing personal stories of how their lives and families have been impacted by the ban, as well as stories read on behalf of families who are afraid to come forward. ” The protest was first  .',
 'Qatar’s sovereign wealth fund has made an iconic purchase in America  —   a stake in the company that owns New York’s Empire State Building. The $  purchase by the Qatar Investment Authority comes as the Doha fund increases its investments in the US as the small country on the Arabian Peninsula tries to cope with low global oil and gas prices. The Empire State Realty Trust Inc. which manages the      building, announced the Qatari purchase late Tuesday, saying the fund would gain a 9.  stake in the company. The trust owns a total of 14 office properties and six retail properties around the New York area. The Qatar Investment Authority did not respond to a request for comment Wednesday. The pointed top of the Art   Empire State Building, once the tallest structure in the world, still stands out in New York’s famed skyline. It remains a major tourist attraction and has been the centerpiece of major American films from “King Kong” to “Sleepless in Seattle. ” Tiny Qatar, an OPEC member, is a strong regional ally for Washington and hosts American bombers and the forward headquarters of the U. S. military’s Central Command at its vast   air base. Aircraft and personnel there are involved in the ongoing   campaign against the Islamic State group in Iraq and Syria. Qatar will host the 2022 FIFA World Cup and has been on a building boom, mirroring on a smaller scale the one that gripped the United Arab Emirates’   of Dubai. However, its government coffers have been hard hit by the drop of global oil prices, which have fallen from over $100 a barrel in the summer of 2014 to around $50 now. The nation’s investment authority, estimated to be worth some $335 billion by the Las   Sovereign Wealth Fund Institute, has been increasingly eyeing opportunities in the US. Last September, it announced plans to open an office in New York and committed to investing $35 billion in the U. S. over the next five years. The fund’s existing American holdings include a more than   stake in New   luxury jeweler Tiffany & Co. It sold its share of the American film studio Miramax to   media group beIN in March for an undisclosed sum.   Qatar Airways, meanwhile, has been rapidly expanding its operations in the US, provoking a backlash from American carriers. Also among the Qatari fund’s interests in America is a   stake in the $8. 6 billion redevelopment project in New York known as Manhattan West, which includes remodeling the building that’s now home to the global headquarters of The Associated Press. The AP announced in August 2015 it planned to move from that building to another near the World Trade Center.',
 'WASHINGTON  —   Score one for MetLife in its fight with the feds to overturn its “   ” label. A federal judge in Washington on Wednesday seemed to side with the life insurance giant by suggesting that the government had it out for the company from the  . MetLife is pushing back against the government’s decision to label it a “systemically important financial institution”  —   a   designation that carries a higher regulatory burden and lumps it in with the big banks. The company, known for using the “Peanuts” character Snoopy in its advertising, is challenging the Financial Stability Oversight Council, a panel of regulators led by Treasury Secretary Jacob Lew, that slapped it with the   in December 2014. In the first major hearing in the case, District of Columbia federal Judge Rosemary Collyer launched into pointed questioning of the government’s position and suggested that the regulatory panel was biased against the company. “There’s nobody neutral in the process,” Collyer said to Eric Beckenhauer, a Department of Justice lawyer representing the government. “They’re all interested. ” The FSOC, which also includes top officials from the Federal Reserve, the Securities and Exchange Commission and other agencies, was created in 2010 under the   financial reform act to weed out risks that could crater the economy again. The “   ” label is one tool it uses to keep closer tabs on big banks and other financial players it deems key to the stability of the markets. MetLife is the only company to sue over the designation, although others are watching the case closely. MetLife claims it doesn’t have the same kind of risks that led to the 2008 collapse of Lehman Brothers or the taxpayer bailout of insurance giant AIG. MetLife’s lawyer, Eugene Scalia, the son of Supreme Court Justice Antonin Scalia, argued that dealing with the FSOC was an opaque process and that it had changed the rules in the middle of the MetLife review. After initially saying it would conduct a risk analysis, the council changed course and wanted to see how vulnerable the firm would be during a severe market downturn. “They changed the rules of the game,” Scalia said during the hearing. The FSOC had “an unfettered imagination to plunge MetLife into financial ruin in order to meet criteria for its material distress. ” Collyer, who isn’t expected to rule on the matter until later this year, seemed sympathetic to Snoopy & Co. “It’s assuming the worst of the worst of the worst,” the judge said.',
 'Looks like Tony Romo is heading to the Super Bowl after all. Sports Business Daily reported Tuesday morning that Romo   replacing former Giants quarterback Phil Simms, and work alongside Jim Nantz starting next season. CBS has the rights to Super Bowl LIII after the 2018 season (NBC has the title game this coming season). News that Romo   came out just hours before the details of the CBS deal were reported. It is believed FOX was offering the    their No. 2 analyst position, replacing John Lynch, who left to take the 49ers GM job. Another former Cowboys quarterback, Troy Aikman, has Fox’s top analyst gig. Simms has been in the booth since 1995, first for NBC and then CBS after it won the rights to broadcast the AFC. It is not known whether Simms will take another football role at CBS or be out altogether. He also works on Showtime’s “Inside the NFL,” but that position may be connected to having one at CBS. “It’s tough to say when I realized I really loved it,”   “And I really think the passion for my job truly grows every year, so that’s good. I never sit around in July and think, ‘Oh my God, this season.’ By   I am ready for it to start. ” SBD reported Romo does not initially want to call both Thursday night and Sunday games, which could be an issue for the NFL because in the past they have insisted on the network’s top broadcast team doing the Thursday package. Those weekday games are split between CBS and NBC; CBS covers the first half of the season. ESPN reported Romo, a scratch golfer,   but it is not in his immediate plans.',
 'A settlement is in the works in a lawsuit against former Yankee Chad Curtis, who’s in a Michigan prison for criminal sexual conduct. A federal judge in Grand Rapids has scheduled a meeting for next Monday. Curtis will participate by video from prison. He’s being sued by four young women who attended Lakewood High School in western Michigan, where he worked after retiring from baseball. They are also suing Lakewood, a school district in Barry County. The women say they were inappropriately touched by Curtis in the weight room. Attorney Monica Beck says her clients are “close to reaching” a deal with him and the district. The former outfielder is serving a seven to 15 year prison sentence after he was   in 2013. The    Curtis played for six teams, including the Yankees, with whom he won two World Series rings in 1998 and 1999. Curtis is eligible for parole in 2020.',
 'One of Google’s   cars was involved in one of the worst autonomous vehicle accident yet, when a driver ran a red light and collided with the passenger side door of the modified Lexus SUV. The driver of the vehicle passed through a red light as one of Google’s fleet of autonomous Lexus SUVs passed through a green light on Friday afternoon. The collision, which occurred at the intersection between El Camino Rea and Phyllis Ave in Mountain View, California, caused the Google car’s airbags to be deployed, and caved in its front and rear   doors. Mountain View police said that no injuries were reported, but the Google car had to be towed away on a flatbed truck. Google’s autonomous vehicles are no stranger to accidents, suffering multiple impacts with various vehicles. Mostly they’ve been   when either driving slowly or stationary at a junction, suffering minor damage. The side impact in this most recent accident is one of only a few that have caused major damage to the expensive test vehicles. Google said that the car was in   mode with a person sitting at the steering wheel. The Google car hit the brakes automatically on seeing the other car crossing the red light, followed by the human behind the wheel doing the same, but it wasn’t enough to prevent the collision. James Allen, who happened upon the crash, told KBCW: “I’ve never seen one in an accident and I see at least 30 to 40 a day. They’re very good cars, that’s why I was so shocked. ” A Google spokesperson told 9to5Google: “Our light was green for at least six seconds before our car entered the intersection. Thousands of crashes happen everyday on US roads, and   running is the leading cause of urban crashes in the US. Human error plays a role in 94% of these crashes, which is why we’re developing fully   technology to make our roads safer. ” Google’s fleet of autonomous cars have covered over 2m miles and has been involved in around two dozen accidents, with only one  —   a collision with a bus  —   being the fault of the   car.',
 'Rihanna takes on the role of Marion Crane in the new series of Bates Motel and the first trailer gives a brief tease of her role. The character was originally played by Janet Leigh in Alfred Hitchcock’s Psycho and later by Anne Heche in Gus van Sant’s    1998 remake. The fifth and final season of the show, which has acted as a contemporary prequel following the character of Norman Bates, will see Crane crop up for a   arc. The character’s background will remain similar, stealing money from her boss so she can be with her boyfriend, but her story will be more fleshed out.  “We’re taking threads of that story and definitely using them so it’s recognizable, it’s just where we go with it is very different,” executive producer Kerry Ehrin said to Entertainment Weekly. “It’s tough to be in a situation where you’re in love with a guy, and for whatever reason, he keeps stalling. You still have all this hot sex with him, and he’s saying he loves you, but he’s stalling. The internal story of that, for a woman, is a really interesting one. ” It’s the first of a set of   roles for Rihanna who will also feature in this summer’s Luc Besson   adventure Valerian and the City of a Thousand Planets and next year’s   caper Ocean’s 8.',
 'American guitarist William Tyler’s excellent fourth album begins with an anxiety attack and ends with an interpolation of Dire Straits’ Sultans of Swing so subtle, I’m still not sure it’s there. The instrumental album strung between those two poles is of such eloquence it renders them superfluous. Context almost gets in the way. Modern Country is a beatific and expansive ambient record daubed in acoustic and electric guitars, analogue oscillations, some really scary bells and no words its meaning can be fluid. But, as Tyler tells it, the album came about when this former member of Lambchop and Silver Jews  —   latterly celebrated for his lyrical acoustic fingerpicking  —  had a panic attack on an interstate. Touring musicians can’t afford such phobias Tyler turned disaster into opportunity, taking the roads less travelled. It was an enforced slowdown soundtracked by country music and the German motorik of the 70s, the sine qua non of driving music. Both traditions, alongside   and Knopfler, find their way onto this lush study of Americana that thrums with a love of the South, a place not synonymous, Tyler argues, with the worst of its past. Kingdom of Jones, a roiling, bright fingerpicked raga, bears witness to Jones County, Mississippi, a historically   enclave. The vanishing America of truck stops and   towns is a cliche now, its bleakness long riffed on in films, photography and song. But Tyler’s parents come from small Southern places, and he returned to a cabin in their university town to write this set of spacious songs, where tradition and transcendence mix. Albion Moonlight reverberates with twang, but behind the guitars, it sounds like someone is working sheet metal next door. The Great Unwind  —   the album’s heroic closer  —   takes its name from New Yorker writer George Packer’s celebrated 2013 account of American inequality. You want to punch the air at the unexpected electric guitar solo. Midway through, there are roughly 30 seconds of undiluted birdsong. Highway Anxiety, meanwhile, betrays no fear, as though Tyler wrote it to soothe himself. The bittersweet melody, the   chord progressions, the sense of acceptance  —   it’s all succour, designed to make your blood pressure drop. Just shy of three minutes in, Wilco drummer Glenn Kotche’s kit rattles into life and Tyler’s awesome band (completed by bassist Darin Gray (Jim O’Rourke, Tweedy) and Justin Vernon associate Phil Cook on keyboards) slip into gear, the austere acoustic framework of Tyler’s previous album, 2013’s also excellent Impossible Truth, receding into the   mirror.',
 'Norwegian police have carried out a controlled explosion of a “  device” in central Oslo and are holding a suspect in custody in an investigation led by security police. Media at the scene described a loud bang early on Sunday shortly after Oslo’s bomb squad arrived with a   robot. The area had been cordoned off by police on Saturday night. “The noise from the blast was louder than our explosives themselves would cause,” a police spokesman said, adding that further investigation was needed to find out if the device had contained explosives. The device, measuring about 1ft (30cm) across, appeared to be capable of causing only a limited amount of damage. Forensics experts will examine fragments to figure out what it was. Police have been on heightened alert since a truck ploughed into a crowd in Stockholm on Friday. Four people were killed and 15 injured in what police called an apparent terrorist attack. Norwegian police detained a suspect but declined to give information about his identity. Norway’s police security service, PST, tweeted that it had taken over the investigation. “We’re in a very early phase of the investigation,” Martin Bernsen, a PST spokesman, said, adding that more details were likely to come later on Sunday. Police took away cordons put up overnight in the Grønland area and residents resumed normal Sunday activities, with shops and cafes open. There was no sign of police at the site. Grønland is a multiethnic neighbourhood that is home to popular bars and restaurants, several mosques, and the city’s main police station. The police station is less than a mile away from where the device was found. In 2011, rightwing extremist Anders Behring Breivik set off a car bomb in Oslo that killed eight people and destroyed Norway’s government headquarters, before going on a shooting rampage that killed 69 people at nearby Utøya island. ',
 'Donald Trump entered a bitter war of words with Senator Jeff Flake on Sunday as the Republican nominee reverted to his behavior in the primary where he constantly attacked elected GOP officials. Trump used Twitter to attack Flake, a   senator from Arizona who has yet to commit to supporting Trump in November, as “very weak and ineffective” on the issue of immigration.    Trump also tweeted: “The Republican Party needs strong and committed leaders, not weak people such as @JeffFlake, if it is going to stop illegal immigration,” after Flake criticized Trump in an interview Sunday with CBS’s John Dickerson.  In the interview, Flake said Trump’s habit of changing his message on immigration was confusing for voters. “It’s kind of a   pivot at times. That’s not clear at all. Some people said it was hardening, some said softening. I say it was just confusing. ”  The result left Flake feeling that it was unlikely that he would be able to support the Republican nominee in the election. “It becomes increasingly difficult to see that he’s going to change, so I don’t expect that I’ll be able to support him in November,” said Flake. Trump had spent the two weeks leading up to his major immigration speech on Wednesday hinting at support for a path to legal status for undocumented immigrants before reversing course. This culminated when Trump made a surprise visit to Mexico to meet with President Enrique Peña Nieto where he described illegal immigration as a “humanitarian crisis”. Only few hours later, while speaking to crowd of supporters in Phoenix, Trump announced “there will be no amnesty” and insisted “anyone who enters the United States illegally is subject to deportation, otherwise we don’t have a country. ” Trump has tangled with Flake before. In July, the Arizona senator confronted Trump in a closed door meeting with other Republican elected officials in Washington DC. Flake has since called on the RNC to shift resources away from Trump and towards downballot races. Trump’s war of words with Flake comes 64 days before the election and with polls in Flake’s home state of Arizona looking increasing close. Hillary Clinton has recently begun advertising in the once deep red state and recent polls show Arizona to be a   in November as the state’s growing Hispanic population has been alienated by Trump’s immigration rhetoric.  In addition, the attack on Flake, who is one of the most prominent Mormon politicians in the United States, risks further alienating that traditionally conservative demographic. Mormons have long viewed the Republican nominee unfavorably and their discontent has contributed to Trump’s weak poll numbers in states like Arizona as well as Utah. Flake is not the only Republican senator who is not voting for the party’s nominee. At least half a dozen others are refusing to back Trump’s candidacy, ranging from   Ben Sasse of Nebraska to moderate Susan Collins of Maine. ',
 'On the internet, we can be anonymous, hitting up a dating site as ”smooth0perator1” in hopes of a   or subtweeting frenemies under the comfort of an inscrutable avatar. In The Internet, Syd Tha Kyd wanted to be anonymous, or at least not the center of attention. It’s one part defense mechanism and one part shruggie emoticon, downplaying her singer status as just someone in the band instead of someone out front. That equal dynamic has served the RB and soul band well, allowing its players to make some wildly funky and thoughtfully layered music as a unit, especially on 2015’s Ego Death. Dropping the honorary used while DJing and engineering Odd Future, Syd steps out like it ain’t no thing with Fin. She makes her initial reservations about going solo seem more like another project, another product. ”This album is not that deep, but I feel like this is my descent into the depth I want the band to get to,” Syd told The Fader in October. ”For me, this is like an   thing  —   maybe get a song on the radio, maybe make some money, have some new s*** to perform.” But it’s clear from the album’s very first moments  —   a digital clang dropped from a cloud of synths, a robotic sigh still so high from a long hit  —   that Syd can smoke out the quiet storm. While she worked with producers   Rakhi,   and fellow Internet member Steve Lacy on Fin, the beats and the words all are driven by Syd. She has an affinity for the ’90s RB singers who kept their emotions open and voices close, like Aaliyah (see ”Know”) or the ladies in TLC (”Smile More,” ”Nothin’ To Somethin’”)  —   embracing the limitations of their range, but finding the core of the performance via   confidence. Syd herself alternates between a chill drawl and a   head voice, but generally keeps everything  . The slithering ”All About Me” has Syd     in a syrup of braggadocio disguised as group support (”People drowning all around me   So I keep my squad around me”) while the sultry ”Body” slinks a woozy   jam. These are the thematic valleys of Fin: hit up your crew when things get rough, and love on your honey with some respect and tenderness. All things considered, that’s not a bad place to be.',
 'As a drought in Venezuela pushes the country’s water levels to extreme lows, President Nicolas Maduro has declared every Friday for the next two months a holiday for public employees to save electricity and water. The government believes the   workweek will help save energy until the rainy season picks up in May, El Pais newspaper reports. According to Newsweek, the South American nation ”depends on hydropower for 60 percent of its electricity.” Maduro’s order, however, excludes   workers, as there is already a shortage of ”grains, meat, dairy and vegetables” due to the economic crisis in Venezuela, Reuters reports. ”Lines of hundreds sometimes snake around supermarkets, so a   work week in that sector would likely have worsened the scarcity,” the news service adds. The   decree is part of a   plan that also asks families, businesses and youth to be aware of their energy usage, Maduro said in a television appearance late Wednesday night. The president’s announcement came after Maduro gave workers three additional days off after the Easter holidays last month. According to Bloomberg, Maduro said ”those efforts saved almost 22 centimeters of water at Guri Dam in the southern state of Bolivar, which supplies as much as 75 percent of the electricity consumed in the capital, Caracas.” According to El Pais, however, the Holy Week shutdown  —   aimed at reducing electricity consumption by 60 percent  —   was a failure, as high temperatures prompted an uptick in the use of air conditioning. The long weekends may appeal to some workers, but not to everyone. ”Just because Maduro doesn’t work Monday to Friday, Saturday or Sunday, doesn’t mean we Venezuelans are like that,” said opposition politician Maria Corina Machado, according to Reuters. ”What we want is to keep working, and for you, Maduro, to go.” Luis Miguel Lopez, who works for the wealthy opposition municipality of Chacao in Caracas, told the news service the decree was ”illogical.” ”People are going to be at home consuming energy all the same,” he said.',
 'Updated at 11 a. m. ET on Dec. 8, Japanese Prime Minister Shinzo Abe will visit Pearl Harbor later this month, becoming the first Japanese head of state to visit the memorial at the site of the surprise attack. President Obama will accompany Abe on the visit, which is scheduled for Dec. 27. The 75th anniversary of the attack is this Wednesday. Earlier this year, Obama became the first sitting U. S. president to visit Hiroshima Peace Memorial Park, where he laid a wreath at a monument to those killed when the U. S. dropped atomic bombs on Hiroshima and Nagasaki. Now, at the USS Arizona Memorial, Abe will honor those killed in the Japanese attack on Pearl Harbor in 1941. ”The two leaders’ visit will showcase the power of reconciliation that has turned former adversaries into the closest of allies, united by common interests and shared values,” the White House said in a statement. The Associated Press has more: ”Abe, in a brief statement, said he would visit Hawaii on Dec. 26 and 27 to pray for the war dead at Pearl Harbor and to hold a final summit meeting with Obama before his presidency ends. ” ’We must never repeat the tragedy of the war,’ he told reporters. ’I would like to send this commitment. At the same time, I would like to send a message of reconciliation between Japan and the U. S.’ . .. ” ’Our talks in Hawaii will be a chance to show the rest of the world our ever stronger alliance in the future,’ Abe said.” The attack on Pearl Harbor in 1941 killed more than 2, 300 American servicemen, pushing the U. S. to enter World War II. Abe has already visited one U. S. memorial to the victims of World War II, as he said in an address to Congress last year. NPR’s Jackie Northam reported at the time: ”Abe told lawmakers that during his visit to Washington, D. C. he went to the National World War II Memorial. He said he stood in silence there for some time. ” ’History is harsh. What is done cannot be undone,’ Abe said, adding, ’I offer with profound respect my eternal condolences to the souls of all American people that were lost during World War II.’ ” NPR, along with many other news outlets, initially reported that Abe would become the first Japanese head of state to visit Pearl Harbor. But in 1951, before the USS Arizona Memorial had been constructed,   Minister Shigeru Yoshida visited Honolulu the AP reported at the time that Yoshida made a brief visit to Pearl Harbor on that trip. Reuters reports that the Japanese government can’t confirm the visit.',
 'Last November, a couple from Washington, D. C. took a weeklong vacation. They visited Mexico, Guatemala and Belize. And got bitten by plenty of mosquitoes. Two days after they returned home, the woman  —   who was pregnant  —   fell ill. She had muscle pain, a fever and a rash. ”At first she didn’t think much about it,” says   Rita Driggers, who saw the woman at Johns Hopkins University School of Medicine. ”But then all the news started coming out about Zika, so the woman went and got tested.” The test came back positive. And then the big question became: Was her baby going to be OK? At first, everything looked good on a standard ultrasound, Driggers says. Even during the 20th week of the pregnancy, the fetus didn’t have microcephaly or calcifications in the brain  —   two telltale signs of a Zika infection. But when doctors looked more closely at the ultrasound and ran MRI on the fetus, the good news quickly faded. ”There were severe abnormalities within the brain,” Driggers says. ”The width of the brain was very, very thin. . .. Some structures were completely missing.” The woman decided to terminate the pregnancy. And she allowed Driggers and her colleagues to study the baby. The case  —   published Wednesday in The New England Journal of Medicine  —   offers insights into how Zika infects a fetus and suggests ways women may be able to find out earlier whether babies will have birth defects. First, the virus lingered in the patient’s blood for months after she got sick.  Usually a person’s immune system clears out Zika in a week or so. But in this case, Driggers thinks the virus was hiding out inside the fetus  —   and repeatedly infecting the mother. ”So if you’re seeing the virus in the mom’s blood more than a week after symptoms,” Driggers says, ”then perhaps what’s going on is that the baby is infected with Zika.” Second, looking only for microcephaly isn’t enough. Right now the Centers for Disease Control and Prevention recommends doctors measure the size of the fetus’s head with an ultrasound near the 20th week of the pregnancy to check for   problems. But with this new case, doctors could see brain abnormalities by ultrasound and MRI before there were signs of microcephaly. ”This is very alarming,” says Dr. Carla Janzen, a   medicine specialist at UCLA, who wasn’t involved in the study. ”If we see more cases like this, the [CDC’s] guidelines will probably have to change.”',
 'The Vatican has issued new guidelines recommending that the cremated remains of Catholics be buried in cemeteries, rather than scattered or kept at home. ”Following the most ancient Christian tradition, the Church insistently recommends that the bodies of the deceased be buried in cemeteries or other sacred places,” state the guidelines released Tuesday by the Vatican. The guidelines do not represent a change the church’s overall policy on burial and cremation, but rather underline ”the doctrinal and pastoral reasons for the preference of the burial of the remains of the faithful and to set out norms pertaining to the conservation of ashes in the case of cremation” in light of the increasing popularity of cremation in many countries, according to the introduction of the document. Cremation has been steadily growing in popularity in the United States. According to the Cremation Association of North America, an industry group for   businesses, nearly half of all people who died in 2015 in the U. S. were cremated, up from about a quarter in 2000. The newly articulated ash norms include not storing human cremains in the home and refraining from scattering ashes ”in the air, on land, at sea or in some other way . .. in order that every appearance of pantheism, naturalism or nihilism be avoided.” The creation of jewelry and other   mementos is also explicitly prohibited by the guidelines. Since its founding, the Roman Catholic Church as an institution has always preferred burial to cremation. For periods, cremation was outlawed entirely. However, since the Second Vatican Council, the official position of the church has been that cremation, while not preferable, is also not banned. The new recommendations reiterate that policy, quoting the church’s canon law in stating: ”The church continues to prefer the practice of burying the bodies of the deceased, because this shows a greater esteem towards the deceased. Nevertheless, cremation is not prohibited, ’unless it was chosen for reasons contrary to Christian doctrine.’ ” Reasons contrary to Christian doctrine, the church says, include ”a denial of Christian dogmas, the animosity of a secret society, or hatred of the Catholic religion and the Church.” ”The Church raises no doctrinal objections to this practice, since cremation of the deceased’s body does not affect his or her soul,” the guidelines continue, ”nor does it prevent God, in his omnipotence, from raising up the deceased body to new life.”',
 ' American Airlines Group Inc ( ) on Tuesday won about $15. 3 million in an antitrust lawsuit that accused airline booking service Sabre Corp ( ) of harming competition and charging grossly inflated booking fees. The Manhattan federal jury awarded nearly $5. 1 million, a fraction of the up to $73 million American Airlines was seeking at trial. But the sum automatically will be tripled under federal antitrust law. American Airlines was suing under the name of US Airways, the carrier it merged with in 2013. US Airways had filed the lawsuit in 2011.  American Airlines welcomed the verdict, saying it hoped the jury’s finding that Sabre had violated federal antritrust law in a 2011 contract with US Airways could result in changes in how the airline’s services are sold. The jury rejected a separate claim that Sabre conspired with its competitors to not compete with each other. Sabre said in a statement that it continued to believe it had operated ”fairly and lawfully.” The company said it would seek to have the verdict set aside and, if unsuccessful, pursue an appeal. Following the verdict, Sabre shares closed at $25. 15, down 35 cents, or 1. 4 percent, on Nasdaq. The case concerned fees that Sabre and other travel reservation systems collect from airlines to display flights for booking. At trial, Chuck Diamond, a lawyer for American Airlines, contended that Sabre used its power in the industry to ”bully” airlines into paying unfair fees and signing unfair contracts that suppress competition and maintain its position. The lawsuit claimed that provisions of a 2011 contract between US Airways and Sabre, including those governing what fares the airline makes available to a computerized network by Sabre used by travel agents, harmed competition. The airline also contended that Sabre conspired with its competitors to not compete with each other for airline content like flight and fare information at the expense of consumers and innovation. Sabre denied conspiring with competitors, and said its contract with US Airways benefited competition. Chris Lind, a lawyer for Sabre, told jurors US Airways was far from powerless as it could leave the network, causing agents to stop using it. The case is US Airways Inc v. Sabre Holdings Corp et al, U. S. District Court, Southern District of New York, No.   .  (Reporting by Nate Raymond in New York; editing by Leslie Adler and Richard Chang) WASHINGTON Federal Reserve policymakers were increasingly split on the outlook for inflation and how it might affect the future pace of interest rate rises, according to the minutes of the Fed’s last policy meeting on June   released on Wednesday. One of the investors former drug company executive Martin Shkreli is accused of defrauding testified on Wednesday that Shkreli lied to him repeatedly, although he eventually made millions of dollars from the investment. ',
 ' A South Korean court approved Italy’s Fincantieri ( ) on Tuesday as the preferred bidder to buy shipbuilder STX France, helping Fincantieri move closer to a deal that could boost its position in the cruise   market. The Seoul court spokesman, who announced the decision, declined to give further details, as did a Fincantieri spokesman. Analysts at Banca IMI estimate Fincantieri would offer below 100 million euros ($104 million) for the asset. The sale of STX France, which specializes in building cruise ships at the   shipyard and is profitable, forms part of the broader sale of businesses from the demise of the South Korean STX shipbuilding group. The French state intends to keep a 33 percent stake in STX France, and French Industry Minister Christophe Sirugue will meet   shipyard unions on Wednesday, said French Finance Minister Michel Sapin. Italy’s   old Fincantieri makes a wide range of vessels from cruise ships to military aircraft carriers, and the former   company had 2015 sales of 4. 2 billion euros. Shares in Fincantieri were up 3. 1 percent at 0. 50 euros in   trading. ”The deal would definitively strengthen Fincantieri’s leadership position in the cruise segment and in the   term we believe it could also enhance the group’s fundamentals,” Banca IMI analysts wrote in a research note last week.  Kepler Cheuvreux also said acquiring STX France would raise Fincantieri’s share of the cruise shipbuilding segment to some 60 percent from   percent. STX France last year delivered the largest passenger ship ever built, the ’Harmony of the Seas’. The holiday cruise industry has recently shown some signs of resilience to headwinds such as the Zika virus scare, that deterred travelers, and currency volatility, with Royal Caribbean Cruises ( ) in October expressing confidence about  meeting its 2017 profit forecast. France has previously said it wants   military shipbuilder DCNS, in which Thales ( ) holds around 35 percent, to take a minority stake in STX France. ”We have strategic interests that we want to keep,” said a DCNS spokesman, commenting on the STX France sale process.  (Additional reporting by Eliza Anzolin in Milan; Writing by Sudip   in Paris; Editing by Alexandra Hudson and Adrian Croft)   Spanish power company Gas Natural  has fired the starting gun in what is expected to be the biggest upheaval among European utility companies for a decade. FRANKFURT Stada’s  chief executive and its head of finance resigned on Tuesday ahead of an imminent new takeover bid for the German generic drugmaker by buyout groups Bain Capital and Cinven. ',
 ' Jurors in the federal hate crimes trial of Dylann Roof watched a video on Friday of the avowed white supremacist confessing to killing nine parishioners at a historic black church in South Carolina and saying he felt he ”had to do it.” Roof told investigators after his arrest for the June 17, 2015, massacre at Emanuel African Methodist Episcopal Church in Charleston that he estimated he had killed five people as retribution for perceived racial grievances. He sounded surprised to learn nine parishioners died. ”I had to do it because somebody had to do it,” Roof said in the taped confession. Asked if he had regrets, Roof said, ”I’d say so, yes . .. I regret that I did it, a little bit.” Roof’s lawyers have not disputed his guilt but hope to spare him from being executed on charges of hate crimes resulting in death, obstruction of religion and firearms violations. Roof, 22, also faces a death sentence if found guilty of murder charges in state court. The videotaped confession, presented on the third day of his federal trial in Charleston, gave jurors a chance to hear the defendant explain why he carried out the attack on a Bible study meeting. He appeared both animated and at ease as he spoke to investigators, laughing at times as he answered their questions. Roof spoke with investigators in Shelby, North Carolina, where he was arrested about 13 hours after security video showed him leaving the church. Inside his car, police said they found a journal where Roof wrote of his dreams for a race war and notes he wrote to his parents. ”Dear Mom, I love you,” read one note presented to jurors. ”I’m sorry for what I did. I know this will have repercussions.” In the video, Roof said white people needed to take a stand against crimes by African Americans. ”I don’t like what black people do,” Roof said, adding he was in favor of reinstating segregation. He chose the Charleston church for the shooting because he knew ”at least a small amount of black people” would be gathered there. Two adults and a child at the Bible study survived. ”It’s like this,” Roof said. ”I’m not in a position, by myself, to go into a black neighborhood and shoot drug dealers.” Nobody ran when he opened fire, he said, and he recalled pausing between shots. ”I was thinking about what I should do,” he said.  (Reporting by Harriet McLeod; Writing by Colleen Jenkins; Editing by Bill Trott and Andrew Hay) WASHINGTON The issuance of U. S. visas, passports and other travel documents should be transferred to the Department of Homeland Security from the State Department, a consulting company commissioned by U. S. Secretary of State Rex Tillerson has recommended in a report. Gene Conley, the only man to win both a baseball World Series and an NBA championship in basketball, died on Tuesday at the age of 86, the Boston Red Sox said in a statement. ',
 ' President Barack Obama on Thursday urged more than 25, 000 volunteers and advocates who dialed in to a White House conference call to pull out the stops to boost the number of people signing up for Obamacare health insurance plans. Obama warned it will be challenging to overcome the skepticism about the plans given an onslaught of headlines about surging premium prices, but he said the stakes are high. ”I think we’re at a critical time where we have to show that this program works for people, if they just see what their options are,” he told the volunteers, who work in their communities to encourage and assist enrollment. Americans who do not receive health insurance through their employer or through Medicare or Medicaid programs shop online for subsidized insurance plans starting Nov. 1 until the end of January. The average premium for benchmark 2017 plans sold on healthcare. gov rose 25 percent compared with 2016. Obama said tax credits will help more than seven in 10 shoppers get a plan for less than $75 per month, but said many may not bother looking because they have heard about spiking costs. ”We’re going to have to clear the bugs off the windshield so people can see the road ahead, and that’s where you guys come in,” Obama said. The law has been fought by Republicans in Congress, who said it creates unwarranted government intervention in personal healthcare and private industry. Several big insurers, including UnitedHealth Group Inc ( ) Aetna Inc ( ) and Humana Inc ( ) are pulling out of the online marketplaces selling the subsidized plans, citing    financial losses. Aetna Chief Executive Mark Bertolini said on Thursday that the earliest his company may return to the marketplaces would be 2019. Obama said more young and healthy people need to sign up for plans. That would offset insurers’ costs of covering members with serious illnesses. Obama has said there are a series of improvements that could be made to his signature domestic policy achievement   the 2010 Affordable Care Act   if Congress and the next president, who will take office on Jan. 20, can work together. ”Part of what we can do this time is to overcome the skeptics, to prove people wrong, and to provide momentum so that when the next administration comes in, they are starting from a position of strength,” Obama told the conference call.  (Reporting by Roberta Rampton; Editing by Jonathan Oatis) MEXICO CITY A meeting between Mexican President Enrique Pena Nieto and U. S. President Donald Trump on Friday at the G20 summit in Germany will last about 30 minutes and probably not lead to any major agreements, Mexico’s foreign minister said on Wednesday. NEW YORK The U. S. government on Wednesday proposed to reduce the volume of biofuel required to be used in gasoline and diesel fuel next year as it signaled the first step toward a potential broader overhaul of its biofuels program. ',
 ' (Reuters Breakingviews)   Americans aren’t just voting for the 45th president. House and Senate elections affect everything from Wall Street to the Supreme Court. And marijuana and healthcare figure prominently in state ballots. Meanwhile, there’s more brouhaha around Brexit. And GE is shaking itself up. LONDON (Reuters Breakingviews)   Worldpay discovered on Wednesday that a couple of birds in the bush are sometimes worth a bit more than one in the hand. A day after Britain’s biggest payment processor said it had received approaches from both Vantiv and JPMorgan, the company plumped for the former’s offer of nearly $10 billion in a mix of cash and shares. That took the shine off the share price of both the buyer and its target. Perhaps because the deal only makes sense with some very charitable HONG KONG Tencent’s   require some Hollywood stardust. The tech giant’s publishing arm may raise up to $800 million in a Hong Kong listing, according to IFR. Catering to bookworms isn’t very lucrative, even if more and more readers are paying for electronic literature. The real payoff will lie in turning stories into blockbuster films, video games and merchandise. ',
 '         The political debate over taxes often centers on income tax  —   a tax that gets increasingly larger for Americans who earn more money. For example, every dollar you make under $9, 275 is taxed at 10 percent  —   but every dollar you make over $415, 050 is taxed at 39. 6 percent. For obvious reasons, this type of tax is less popular with   Americans. But there are other federal taxes that Americans pay, too, the biggest of which are payroll taxes. Whether you earn $5, 000 or $100, 000, Americans pay up to 15. 3 percent of their income to the government  —   largely to support Social Security and Medicare. (Some payroll taxes are levied on your employer, who is likely to pass down some or all of it to you. This is why the amount you are on the hook for is tough to pin down.) And income over $118, 500 isn’t subject to Social Security taxes, which means those who earn more start paying a smaller percentage of their income into payroll taxes.    Politicians don’t spend much time debating the payroll taxes, and most of us don’t spend much time thinking about them, either. In fact, most people don’t even search the internet for the taxes on their pay stub called ”Medicare” and ”Social Security,” which are payroll taxes.    Here’s a chart that shows how the terms compare, in terms of Google search volume:           Payroll taxes don’t factor into the political debate as much as income taxes, but they should: Most   families pay more in payroll taxes than they do in income tax. Having an income tax —  dominated debate presents a skewed picture of how all taxpayers fund the government. My colleague Dylan Matthews writes:     The taxes the privileged pay [end up] worthy of attention and the ones the poor pay are ignored. It paints a picture where the government is being supported on the backs of the wealthy, and the poor and middle class are  . It leads to plans for various kinds of tax cuts and tax reforms that matter massively for the rich and very little for the poor.  That’s why we charted out historical tax rates in the interactive above. You see that payroll taxes have been rising steadily while the top rates for income taxes have come down quite a bit. But as the search data shows, there just isn’t much talk about the taxes that put the biggest burden on the middle class. An earlier version of this story did not point out that income over $118, 500 isn’t subject to Social Security taxes.     What’s the payroll tax?     How candidate’s tax plans could affect your wallet    ',
 ' The Supreme Court’s decision last June legalizing   marriage across the country set up an odd puzzle for LGBTQ people in America. Though everyone can now legally marry the person of their choosing, in many states individuals can still be fired or otherwise discriminated against purely on the basis of their sexual orientation or gender identity.    The barrier isn’t a religious liberty law, like the one passed to widespread scorn in Indiana last March. Instead, 30 states just don’t include ”sexual orientation” and ”gender identity” as protected categories under their respective civil rights laws. That’s still the case in Florida, where the state’s legislature tried but failed to pass a law banning LGBTQ discrimination by adding sexual orientation and gender identity to the state’s civil rights law. The bill, backed by both Democrats and Republicans, along with numerous   businesses in the state, died yesterday in the state Senate Judiciary Committee, where it was being considered. In fairness, the bill did garner a public hearing  —   unlike the nine previous times similar bills were proposed.    Go deeper:  Read Vox’s German Lopez on why so many states haven’t updated their civil rights laws  —   and why more people don’t know about the problem. The Human Rights Campaign mapped the bizarre mix of LGBTQ laws by state.  ',
 ' In an exclusive interview with Time’s Michael Scherer, published Thursday, President Donald Trump basically gives three types of explanations for why it’s okay for him to say things that aren’t true.     Trump thinks it’s okay to repeat a false claim as long as he is accurately stating the fact that someone else made the false claim. He thinks it’s okay to say something false as long as it later turns out that something vaguely similar is true. And he appears to think it’s okay to just   lie and then keep lying for no reason.  Here are three quotes on category one misstatements, where Trump says he was accurately saying other people said something false, so there’s no reason for him to apologize for spreading misinformation:  “I’m just quoting the newspaper [about Ted Cruz’s father being involved in the JFK assassination] just like I quoted the judge the other day, Judge Napolitano, I quoted Judge Napolitano, just like I quoted Bret Baier, I mean Bret Baier mentioned the word wiretap. ” “Well, I’m not, well, I think, I’m not saying, I’m quoting, Michael, I’m quoting highly respected people and sources from major television networks. ” “Well if you look at the reporter, he wrote the story [about Muslims celebrating   on rooftops in New Jersey] in the Washington Post. ”  Here are two quotes on category two misstatements where Trump says it’s okay to say something false, because something different would later happen that was kind of similar to the thing he falsely said earlier:     “Sweden. I make the statement, everyone goes crazy. The next day they have a massive riot, and death, and problems. ” “Because a wiretapping is, you know today it is different than wire tapping. It is just a good description. But wiretapping was in quotes. What I’m talking about is surveillance. And today, [House Intelligence Committee Chair] Devin Nunes just had a news conference. ”  And here are four quotes on category three misstatements, where Trump simply insists that false claims he’s made in the past are, in fact, true.  “Huma [Abedin] and Anthony [Weiner] you know, what I tweeted about that whole deal, and then it turned out he had it, all of Hillary’s email on his thing. ”  “NATO, obsolete, because it doesn’t cover terrorism. They fixed that, and I said that the allies must pay. ” “I inherited a mess with jobs, despite the statistics, you know, my statistics are even better, but they are not the real statistics because you have millions of people that can’t get a job, ok. ”  At the end, Trump closes with one last excuse, the ultimate Trump rationale for anything he does  —   he won the election: “I can’t be doing so badly, because I’m president, and you’re not. You know. ” ',
 ' About one year ago, Bernie Sanders memorably told Hillary Clinton during a primary debate that the American people were “sick and tired of hearing about your damn emails. ”    It appears he still thinks they are. In an interview with the Washington Post published on Monday, Sanders downplayed talk that the Clinton campaign had disrespected the Vermont senator in internal emails hacked by WikiLeaks. (Some of the emails showed Clinton allies cursing at Sanders endorsers and referring to his proposals as emerging from a “leftie alternative universe. ”) “Trust me, if they went into our emails  —   I suppose which may happen, who knows  —   I’m sure there would be statements that would be less than flattering about, you know, the Clinton staff,” Sanders told the Post. “That’s what happens in campaigns. ” Sanders has a point. One of the challenges in evaluating WikiLeaks’ massive dump of the emails from John Podesta, Clinton’s campaign chair, is that we don’t really have anything to compare it to. There is no trove of the emails from Donald Trump campaign manager Kellyanne Conway (or Corey Lewandowski or Paul Manafort). We do not have the internal exchanges of Jeff Weaver, Sanders’s campaign manager  —   though, as Sanders suggests, they might be fascinating to see.    Sanders on Clinton’s team: “The way they do politics is very different” The Podesta emails have revealed a lot about Clinton’s campaign operation. They show that her speeches and positions were subject to extensive internal debates, and suggest how crass political considerations often influenced her ultimate positions.    That’s certainly interesting, but it’s hard to contextualize. In the absence of a similar trove of leaked internal emails, it’s impossible to know what here reflects normal political maneuvering  —   and what represents something genuinely unique to Clinton’s team. In his interview with Wagner, Sanders says that the Podesta emails don’t reveal anything disqualifying about Clinton. But he also hints that her campaign had an operation whose internal process differed dramatically from his, appearing to cite an exchange in which a   political staffers debated whether to include a joke about her emails.    Here’s Sanders:  The way they work is very, very different than the way we work. We did not have a committee deciding what kind of jokes I would be telling. In fact, we usually had me scrambling to write my speech on a yellow piece of paper, which I finished three minutes before I would go up there They were much more prepared and much better organized and careful about what they were saying or not saying.   …     The way they do politics is very different.  Since endorsing Clinton in July, Sanders has praised Clinton for agreeing to a progressive platform, papered over their policy disagreements, and admitted that she won “fair and square. ” But old wounds don’t heal overnight, and Sanders still clearly believes he was playing on an uneven playing field in the Democratic primary. ',
 ' Five paragraphs into the Wall Street Journal’s article about how Carl Icahn, a legendary investor worth about $20 billion, will serve as a special advisor to Donald Trump on regulatory matters, things get interesting.    “The position isn’t an official government job,” the Journal reports. “Mr. Icahn won’t get paid and won’t have to give up his current business dealings. ” Of course, even if Icahn did get paid, the salary would be peanuts compared with his net worth. More to the point, the monetary value of getting to influence federal regulatory policy when you already have $20 billion in outstanding investments is enormous. Back in August, for example, Icahn was complaining to the media about a particular obscure Environmental Protection Agency rule that was hurting a refining company he owns.  Complaining to the media about regulatory decisions you don’t like is, of course, as American as apple pie. So is hiring a lobbyist or three to press your case. But to have an active businessman formally in a position to do the regulatory work is a fairly open invitation to corruption.     And that’s the kind of thing that past administrations have tried to avoid. But since Donald Trump himself is tied up in unprecedented conflicts of interest, that apparently lowers the bar for the rest of his team too.  ',
 '     The Senate Intelligence Committee, which is investigating Russian interference in the 2016 presidential race, has asked President Trump’s political organization to gather and produce all   documents, emails and phone records going back to his campaign’s launch in June 2015, according to two people briefed on the request. The letter from the Senate arrived at Trump’s campaign committee last week and was addressed to the group’s treasurer. Since then, some former staffers have been notified and asked to cooperate, the people said. They were not authorized to speak publicly. The demand follows a Senate request months earlier for the campaign committee to preserve documents. Dozens of former staffers are expected to be contacted in the coming days to make sure they are aware of what they are required to produce and how to submit those documents, the people added. The letter was signed by Sen. Richard Burr ( . C.) the Senate committee’s chairman, and Sen. Mark R. Warner ( .) the committee’s ranking Democrat. Spokespeople for Burr and Warner declined to comment.       The Washington Post’s Devlin Barrett explains the Justice Department’s decision to appoint Robert Mueller as special counsel to investigate possible connections between the Trump campaign and Russian officials. (Peter Stevenson, Jason Aldag, Whitney   Washington Post)   The request to Trump’s political operatives represents the first time that Trump’s official campaign structure has been drawn into the Senate committee’s ongoing bipartisan investigation. That investigation is separate from the federal probe being led by the Justice Department’s special counsel, former FBI director Robert S. Mueller III. In recent months, several Trump campaign associates, such as Roger Stone and Carter Page, have been contacted by Senate investigators, but the campaign itself had not been asked to preserve and produce materials. Trump’s campaign committee is now led by former deputy campaign manager Michael Glassner and John Pence, a nephew of Vice President Pence. It is based in New York at Trump Tower. Glassner did not immediately respond to a request for comment. A White House spokesperson had no immediate comment. John Wagner and Karoun Demirjian contributed to this report.',
 '  CAIRO  —   The photos of the Sharef family spoke volumes about their plight. In the first two, the Iraqis are happily seated on their plane, smiling. They were flying from their home in Irbil to New York. In the next few, they are seated in Cairo’s airport, their faces glum and haggard. By then, they had been taken off their plane  —   and informed they could no longer travel to the United States. It did not matter that they had valid visas. It did not matter that they were headed to Nashville to start a new life. President Trump’s executive order banning entry to citizens of Iraq and six other mostly Muslim nations had caught up with the family of five. “I am a very hard worker,” Fuad Sharef, the father, said in a telephone interview because they were not allowed to leave the airport terminal. “Going to America was a dream for me and my kids. “Everything has gone down the drain because of Donald Trump. ”      (Reuters)   Down the drain means this: The family had sold their house, their car and all their possessions to aid them in their new life. The children were pulled out of their schools. Sharef quit his   job at a pharmaceutical company. And their air tickets cost $5, 000. Also down the drain is their sense of security. Sharef once worked for a U. S. government subcontractor in   Iraq as a translator and a program manager. He got his visas, after two years of vetting, through a special U. S. resettlement program for Iraqi employees of the American government. Working for Americans was filled with perils, he said. He and other colleagues faced death threats; he knew   who were kidnapped or killed. On Sunday, he and his family  —   his wife, Arazoo, 41; his son, Bnyad, 19; his daughter Yad, 17; and another daughter, Shad, 10  —   boarded a flight back to Irbil after spending the night inside the airport terminal. “Donald Trump destroyed my life,” Sharef said. “How can he do this to people who risked their lives to help America?”  Read more:              ',
 '     For students across the country, the traditional   trip to Washington is a chance to join the throngs on the Mall and perhaps spot some of the world’s most powerful people on the grounds of the U. S. Capitol. But a group from South Orange Middle School in New Jersey may remember their trip to the nation’s capital last week for another reason: It was the occasion for a pointed snub of House Speaker Paul D. Ryan ( . ). Dozens of the 218 students on the trip refused to have their photo taken with Ryan when he briefly joined them outside the capitol Thursday, students on the trip said. Those present were unable to provide a precise tally of how many opted out.      Matthew Malespina, one of the students who stayed away, said in an interview Sunday that he chose not to be photographed with Ryan because he disagreed with the policies the speaker and his party are pushing on health care, among other things. He called Ryan “a man who puts his party before his country. ” He said he and the others stood across the street while Ryan posed with their peers. Their act of civil disobedience was picked up by a local news website, the Village Green, and drew attention from larger media outlets. “I don’t like to take a picture with somebody that I can’t associate with,” Matthew, 13, told The Washington Post. “Let’s say somebody is not nice to me at school, for example. I wouldn’t take a picture with them, probably. ”    House Speaker Paul D. Ryan ( .) speaks at a news conference Thursday on Capitol Hill in Washington. (Andrew  )  Sophia Kraiker, 14, another student who declined to be in the photo, told The Post that she wanted to distance herself from Ryan because he is “shadowing Trump’s ideas. ” Asked about the incident Sunday, a spokesman for Ryan said the speaker “always appreciates the opportunity to welcome students to the Capitol. ” Ryan posted a picture of himself on Instagram on Friday giving a   to one of the students in the group who agreed to pose. Matthew’s mother, Elissa Malespina, a   librarian who was not on the trip, said she was surprised but pleased to hear of the students’ protest action. She posted about it on her Facebook page.  “I’m proud of him, and I’m proud of the other students that chose to exercise their constitutional rights and did so in a respectful manner,” Malespina said Sunday. Although little blowback has come their way in the liberal suburb of South Orange, N. J. Malespina said some of the online comments on news stories about the students’ actions have been vitriolic, often focusing on the parents. She said that while politics are a frequent topic of discussion in their household, criticism that she or other parents “indoctrinated” the kids involved is unfounded. “Teenagers, honestly, do they listen to their parents anyway?”  ',
 '     Weeks before he takes the oath of office,   Donald Trump is scheduled for another kind of oath  —   he’s set to face lawyers’ questions during a deposition in the $10 million    lawsuit he filed against   celebrity chef José Andrés. In filings last week, attorneys for Trump and Andrés’s company argued over the details of the session slated to take place the first week of January. Trump wanted the deposition to take place in New York, where he lives and where his presidential transition is based, not in Washington, for security reasons. Andrés’s lawyers agreed. But Trump’s team has other objections: The real estate executive wants to limit questioning to two hours and prohibit questions covered in a previous deposition. “It seems dubious that the   cannot be afforded adequate security in the capital of the United States, but defendants are willing to accommodate that demand,” Andrés’s attorneys wrote in a filing. “Defendants cannot, however, accept Trump LLC’s attempt to hamstring defendant’s questioning of the man who directed the bringing of this lawsuit. ”      D. C.    José Andrés says he is backing out of a deal to launch a restaurant in the hotel Republican presidential candidate Donald Trump is building in the historic Old Post Office Pavilion. (WUSA9)   The   legal battle began when the chef backed out of a deal to open a restaurant in Trump’s luxury hotel complex located in the Old Post Office on Pennsylvania Avenue. Andrés cited Trump’s derogatory comments on the campaign trail about illegal immigrants, saying they made it impossible for him to run a successful eatery. Trump promptly sued. Trump was deposed in June in a similar lawsuit against restaurateur Geoffrey Zakarian, who was also slated to run a restaurant in Trump’s Washington hotel and  —   like Andrés  —   scrapped the plan after Trump’s incendiary rhetoric created controversy. Zakarian’s case is apace, too: His attorneys on Monday set a hearing date of Jan. 3 in D. C. Superior Court.',
 '     Trivia fans: What do the performers of “Gypsys, Tramps & Thieves,” “Evergreen” and “I Feel the Earth Move” have in common? Two things: All three scored No. 1 hits in the 1970s. And about four decades later, the three women  —   Cher, Barbra Streisand and Carole King  —   have composed fundraising pitches on behalf of Hillary Clinton and fellow Democrats. On Tuesday, King put forward the latest solicitation, asking donors to help elect Democrats to the U. S. House so that “Hillary’s presidency lives up to its potential. ”   [Inside the exclusive events helping to fund Clinton and the Democratic Party]   Earlier this month, Cher  —   who previously pledged to move to Jupiter if Republican Donald Trump is elected president  —   sent an   email declaring: “I’m Proud 2 Support Hillary! !” And Streisand has sent emails to Clinton supporters to entice them to try to win a seat at a concert she plans to benefit Clinton next month in New York. Both Cher and King also have put their enduring musical talents to work for Clinton. Cher recently appeared at a Clinton fundraiser in Provincetown, Mass. while King, who met Clinton when she was first lady, performed “You’ve Got a Friend” at the Democratic National Convention in July. Clinton, 68, has also engaged younger performers  —   pop star Katy Perry has joined her several times  —   but the Democratic nominee seems to have a particular fondness for artists from her generation. Entertainment was provided at a major Clinton fundraiser Tuesday night in the Hamptons by Jimmy Buffett and Paul McCartney  —   both within Clinton’s age range  —   as well as Jon Bon Jovi. During a recent campaign stop in Cleveland, Clinton diverted her motorcade to the arena where McCartney was playing that night for a chance to schmooze with the former Beatles member. Meanwhile, Paul Simon, who split with Art Garfunkel in 1970, belted out “Bridge Over Troubled Water” at the Democratic convention.']

In [10]:
publication_rand = ['New York Times',
 'New York Times',
 'New York Times',
 'New York Times',
 'New York Times',
 'Breitbart',
 'Breitbart',
 'Breitbart',
 'Breitbart',
 'Breitbart',
 'CNN',
 'CNN',
 'CNN',
 'CNN',
 'CNN',
 'Business Insider',
 'Business Insider',
 'Business Insider',
 'Business Insider',
 'Business Insider',
 'Atlantic',
 'Atlantic',
 'Atlantic',
 'Atlantic',
 'Atlantic',
 'Fox News',
 'Fox News',
 'Fox News',
 'Fox News',
 'Fox News',
 'Talking Points Memo',
 'Talking Points Memo',
 'Talking Points Memo',
 'Talking Points Memo',
 'Talking Points Memo',
 'Buzzfeed News',
 'Buzzfeed News',
 'Buzzfeed News',
 'Buzzfeed News',
 'Buzzfeed News',
 'National Review',
 'National Review',
 'National Review',
 'National Review',
 'National Review',
 'New York Post',
 'New York Post',
 'New York Post',
 'New York Post',
 'New York Post',
 'Guardian',
 'Guardian',
 'Guardian',
 'Guardian',
 'Guardian',
 'NPR',
 'NPR',
 'NPR',
 'NPR',
 'NPR',
 'Reuters',
 'Reuters',
 'Reuters',
 'Reuters',
 'Reuters',
 'Vox',
 'Vox',
 'Vox',
 'Vox',
 'Vox',
 'Washington Post',
 'Washington Post',
 'Washington Post',
 'Washington Post',
 'Washington Post']

# Pegasus for Generic Summarization

In [11]:
#A lot of the stuff got imported from the libraries up top.
#If anything else needs to be imported ill do so here
#Importing the pegasus model and tokenizer
from transformers import PegasusTokenizer, TFPegasusForConditionalGeneration
from transformers import AutoConfig
from transformers import PegasusConfig, PegasusModel

#Getting the model
pmodel = TFPegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")
pmodel.config.max_position_embeddings = 1024
ptokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")
#pmodel = TFPegasusForConditionalGeneration.from_pretrained("google/pegasus-large")
#ptokenizer = PegasusTokenizer.from_pretrained("google/pegasus-large")

#Have to change this otherwise won't work
#pmodel.config.max_position_embeddings = 1024

#Rouge evaluator
rouge = evaluate.load('rouge')

pmodel.config

#These are the current number of publications
#data.publication.value_counts()

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFPegasusForConditionalGeneration.

Some layers of TFPegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.52M [00:00<?, ?B/s]

PegasusConfig {
  "_name_or_path": "google/pegasus-xsum",
  "activation_dropout": 0.1,
  "activation_function": "relu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "PegasusForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 16,
  "decoder_start_token_id": 0,
  "do_blenderbot_90_layernorm": false,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 16,
  "eos_token_id": 1,
  "extra_pos_embeddings": 0,
  "force_bos_token_to_be_generated": false,
  "forced_eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,


In [12]:
publications_unique = data.publication.unique()
publications_unique

array(['New York Times', 'Breitbart', 'CNN', 'Business Insider',
       'Atlantic', 'Fox News', 'Talking Points Memo', 'Buzzfeed News',
       'National Review', 'New York Post', 'Guardian', 'NPR', 'Reuters',
       'Vox', 'Washington Post'], dtype=object)

Run this to check parameters

In [35]:
#This is where we will store the summaries
generated_summary = []

#Rouge results
rogue_results = []

In [36]:
#Generate summaries

#Token range parameter. How much bigger (or smaller, but usually bigger) can the summary be? 5 seems to be a good number. probably won't change this.
token_range = 10
#Number of beams. How "deep" does the model look at the next word (token I think) before generating. 4 seems to be good and stable. Headlines are not that big.
number_of_beams_parameter = 4
#No repeat n gram.
no_repeat_ngram_size_parameter = 4

for i in range(75):
  #Pegasus does not need a task prompt
  ARTICLE_TO_SUMMARIZE = articles_rand[i]
  REFERENCE = headlines_rand[i]

  #Tokenizing the input, aka prepping to put it into BERT
  #inputs = ptokenizer(ARTICLE_TO_SUMMARIZE, max_length=1024, truncation=True, return_tensors="tf")
  inputs = ptokenizer(ARTICLE_TO_SUMMARIZE, max_length=512, truncation=True, return_tensors="tf")

  #Generating summary
  summary_ids = pmodel.generate(inputs["input_ids"],
                                num_beams=number_of_beams_parameter,
                                max_length=len(ptokenizer(REFERENCE)['input_ids']) + token_range,
                                min_length=len(ptokenizer(REFERENCE)['input_ids']) - token_range,
                                no_repeat_ngram_size=no_repeat_ngram_size_parameter
                                )
  #Decoding
  candidate = ptokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)

  predictions = candidate
  references = [REFERENCE]
  results = rouge.compute(predictions=predictions,
                          references=references)
  rogue_results.append(results)

  print(str(i), " done")


0  done
1  done
2  done
3  done
4  done
5  done
6  done
7  done
8  done
9  done
10  done
11  done
12  done
13  done
14  done
15  done
16  done
17  done
18  done
19  done
20  done
21  done
22  done
23  done
24  done
25  done
26  done
27  done
28  done
29  done
30  done
31  done
32  done
33  done
34  done
35  done
36  done
37  done
38  done
39  done
40  done
41  done
42  done
43  done
44  done
45  done
46  done
47  done
48  done
49  done
50  done
51  done
52  done
53  done
54  done
55  done
56  done
57  done
58  done
59  done
60  done
61  done
62  done
63  done
64  done
65  done
66  done
67  done
68  done
69  done
70  done
71  done
72  done
73  done
74  done


In [37]:
import statistics
rouge1 = []
rouge2 = []
rougeL = []
rougeLsum = []

In [38]:
for i in rogue_results:
  rouge1.append(i['rouge1'])
  rouge2.append(i['rouge2'])
  rougeL.append(i['rougeL'])
  rougeLsum.append(i['rougeLsum'])

In [39]:
#rogue_results

In [40]:
print("Variance of Rouge1: ", np.var(rouge1))
print("Variance of Rouge2: ", np.var(rouge2))
print("Variance of RougeL: ", np.var(rougeL))
print("Variance of RougeLSum: ", np.var(rougeLsum))

print("Median of Rouge1: ", statistics.median(rouge1))
print("Median of Rouge2: ", statistics.median(rouge2))
print("Median of RougeL: ", statistics.median(rougeL))
print("Median of RougeLSum: ", statistics.median(rougeLsum))

print("Average of Rouge1: ", sum(rouge1)/len(rouge1))
print("Average of Rouge2: ", sum(rouge2)/len(rouge2))
print("Average of RougeL: ", sum(rougeL)/len(rougeL))
print("Average of RougeLSum: ", sum(rougeLsum)/len(rougeLsum))

print("Max of Rouge1: ", max(rouge1))
print("Max of Rouge2: ", max(rouge2))
print("Max of RougeL: ", max(rougeL))
print("Max of RougeLSum: ", max(rougeLsum))

print("Min of Rouge1: ", min(rouge1))
print("Min of Rouge2: ", min(rouge2))
print("Min of RougeL: ", min(rougeL))
print("Min of RougeLSum: ", min(rougeLsum))

rougeL.index(max(rougeL))

Variance of Rouge1:  0.024324209910715204
Variance of Rouge2:  0.011746673500715198
Variance of RougeL:  0.020852347365081868
Variance of RougeLSum:  0.020852347365081868
Median of Rouge1:  0.21428571428571425
Median of Rouge2:  0.06666666666666667
Median of RougeL:  0.18181818181818182
Median of RougeLSum:  0.18181818181818182
Average of Rouge1:  0.2319769868042693
Average of Rouge2:  0.08374658972198096
Average of RougeL:  0.2013279718471741
Average of RougeLSum:  0.2013279718471741
Max of Rouge1:  0.7777777777777777
Max of Rouge2:  0.5
Max of RougeL:  0.7777777777777777
Max of RougeLSum:  0.7777777777777777
Min of Rouge1:  0.0
Min of Rouge2:  0.0
Min of RougeL:  0.0
Min of RougeLSum:  0.0


11

In [41]:
headlines_rand

['Donald Trump Takes Credit for Helping to Save a Ford Plant That Wasn’t Closing',
 '‘Ring of Fire’ Eclipse Travels Across South America and Africa',
 'Bruce Springsteen’s Archive Is Headed to Monmouth University',
 'California Official Says Trump’s Claim of Voter Fraud Is ‘Absurd’',
 'Monkey in Kenya Survives After Setting Off Nationwide Blackout',
 'Hillary Clinton Warns of ‘Conspiracy Theory Machine Factory’',
 'Report: GOP Differences on Healthcare ’Have Narrowed,’ Healthcare Reform ’Very Much Alive’',
 'Donald Trump Appoints Top Deputies, More White House Staff',
 'LISTEN: Green Day Single Focuses on American ’Culture of Mass Shooting’',
 'Five Facts About Donald Trump’s Expected Pick for Interior Secretary, Rep. Cathy McMorris Rodgers',
 'George W. Bush to attend fundraiser in support of Gillespie',
 'Justin Timberlake and Jessica Biel expecting first baby',
 'Bill Cosby drops lawsuit against accuser',
 'Lupita Nyong’o’s dress possibly recovered',
 'New Trump accuser: GOP nominee

In [ ]:
print(headlines_rand[11])

#Tokenizing the input, aka prepping to put it into BERT
inputs = t5tokenizer("summarize: " + articles_rand[11], max_length=1024, truncation=True, return_tensors="tf")

#Generating summary
summary_ids = t5model.generate(inputs["input_ids"],
                               num_beams=number_of_beams_parameter,
                               max_length=len(t5tokenizer(REFERENCE)['input_ids']) + token_range,
                               min_length=len(t5tokenizer(REFERENCE)['input_ids']) - token_range,
                               no_repeat_ngram_size=no_repeat_ngram_size_parameter
                               )
#Decoding
candidate = t5tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(candidate[0])

Justin Timberlake and Jessica Biel expecting first baby
Justin Timberlake has confirmed that his wife, Jessica Biel, is expecting . the singer said they don't know the sex of the baby yet
